## Data Preprocessing

### Calculate the VAFs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from IPython.display import display, HTML

# Set directories
DATA_DIR = 'data'
RESULTS_DIR = 'results'

def parse_strelka2_vcf(vcf_file, sample_name=None, patient_id=None):
    variants = []
    
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
                
            fields = line.strip().split('\t')
            
            if len(fields) < 10:
                continue
                
            chrom = fields[0]
            pos = int(fields[1])
            ref = fields[3]
            alt = fields[4]
            info = fields[7]
            format_fields = fields[8].split(':')
            normal_values = fields[9].split(':')
            tumor_values = fields[10].split(':')
            
            normal_data = dict(zip(format_fields, normal_values))
            tumor_data = dict(zip(format_fields, tumor_values))
            
            nucleotide_to_field = {'A': 'AU', 'C': 'CU', 'G': 'GU', 'T': 'TU'}
            
            try:
                ref_counts_index = format_fields.index(nucleotide_to_field[ref])
                alt_counts_index = format_fields.index(nucleotide_to_field[alt])
            except (ValueError, KeyError):
                continue
                
            tumor_ref_counts = tumor_values[ref_counts_index].split(',')
            tumor_alt_counts = tumor_values[alt_counts_index].split(',')
            
            tier1_ref_counts = int(tumor_ref_counts[0])
            tier1_alt_counts = int(tumor_alt_counts[0])
            
            total_counts = tier1_ref_counts + tier1_alt_counts
            vaf = tier1_alt_counts / total_counts if total_counts > 0 else 0
            
            tumor_depth = int(tumor_data.get('DP', 0))
            is_somatic = "SOMATIC" in info
            
            variant_data = {
                'PATIENT_ID': patient_id,
                'SAMPLE': sample_name,
                'CHROM': chrom,
                'POS': pos,
                'REF': ref,
                'ALT': alt,
                'REF_COUNT': tier1_ref_counts,
                'ALT_COUNT': tier1_alt_counts,
                'TOTAL_COUNT': total_counts,
                'VAF': round(vaf, 4),
                'TUMOR_DEPTH': tumor_depth,
                'SOMATIC': is_somatic
            }
            
            variants.append(variant_data)
    
    variants_df = pd.DataFrame(variants)
    return variants_df

def find_vcf_files(data_dir):
    vcf_files = []
    
    for patient_dir in os.listdir(data_dir):
        patient_path = os.path.join(data_dir, patient_dir)
        
        if os.path.isdir(patient_path):
            patient_id = patient_dir
            
            for sample_dir in os.listdir(patient_path):
                sample_path = os.path.join(patient_path, sample_dir)
                
                if os.path.isdir(sample_path):
                    sample_id = sample_dir
                    
                    for filename in os.listdir(sample_path):
                        if filename.endswith("snvs.pass.vcf"):
                            vcf_path = os.path.join(sample_path, filename)
                            
                            vcf_files.append({
                                'patient_id': patient_id,
                                'sample_id': sample_id,
                                'vcf_path': vcf_path
                            })
    
    return vcf_files

def create_manhattan_plot(data, by_sample=True, output_dir=None, patient_id=None):
    multiple_samples = 'SAMPLE' in data.columns and len(data['SAMPLE'].unique()) > 1
    
    plt.figure(figsize=(15, 6))
    
    data_copy = data.copy()
    data_copy['CHROM_CLEAN'] = data_copy['CHROM'].str.replace('chr', '')
    
    def chrom_to_num(chrom):
        if chrom in ['X', 'Y', 'M']:
            return {'X': 23, 'Y': 24, 'M': 25}[chrom]
        try:
            return int(chrom)
        except ValueError:
            return 999
    
    data_copy['CHROM_NUM'] = data_copy['CHROM_CLEAN'].apply(chrom_to_num)
    data_copy = data_copy.sort_values(['CHROM_NUM', 'POS'])
    
    chromosomes = data_copy['CHROM'].unique()
    chrom_sizes = {}
    
    for chrom in chromosomes:
        chrom_data = data_copy[data_copy['CHROM'] == chrom]
        if not chrom_data.empty:
            chrom_sizes[chrom] = chrom_data['POS'].max()
    
    chrom_pos = {}
    cum_pos = 0
    
    for chrom in sorted(chromosomes, key=lambda x: data_copy[data_copy['CHROM'] == x]['CHROM_NUM'].iloc[0]):
        chrom_pos[chrom] = cum_pos
        cum_pos += chrom_sizes.get(chrom, 0) + 5000000
    
    data_copy['GENOME_POS'] = data_copy.apply(lambda row: chrom_pos[row['CHROM']] + row['POS'], axis=1)
    
    if multiple_samples and by_sample:
        samples = data_copy['SAMPLE'].unique()
        palette = sns.color_palette("tab10", n_colors=len(samples))
        sample_colors = dict(zip(samples, palette))
        
        for sample, color in sample_colors.items():
            sample_data = data_copy[data_copy['SAMPLE'] == sample]
            plt.scatter(sample_data['GENOME_POS'], sample_data['VAF'], 
                      color=color, alpha=0.7, s=10, label=sample)
        
        plt.legend(title='Sample', loc='upper right')
    else:
        for i, chrom in enumerate(sorted(chromosomes, key=lambda x: data_copy[data_copy['CHROM'] == x]['CHROM_NUM'].iloc[0])):
            chrom_data = data_copy[data_copy['CHROM'] == chrom]
            if not chrom_data.empty:
                plt.scatter(chrom_data['GENOME_POS'], chrom_data['VAF'], 
                          color=f'C{i % 10}', alpha=0.7, s=10, label=chrom)
    
    chrom_midpoints = {}
    for chrom in chromosomes:
        chrom_data = data_copy[data_copy['CHROM'] == chrom]
        if not chrom_data.empty:
            midpoint = chrom_pos[chrom] + chrom_sizes.get(chrom, 0) / 2
            chrom_midpoints[chrom] = midpoint
    
    plt.xticks([midpoint for midpoint in chrom_midpoints.values()], 
              [chrom for chrom in chrom_midpoints.keys()],
              rotation=45)
    
    plt.ylabel('VAF')
    plt.title('VAF Distribution Across the Genome')
    plt.axhline(y=0.5, color='r', linestyle='--', label='VAF = 0.5')
    
    if not (multiple_samples and by_sample):
        handles, labels = plt.gca().get_legend_handles_labels()
        if len(handles) > 10:
            handles = handles[:10]
            labels = labels[:10]
            labels[-1] += "..."
        
        plt.legend(handles, labels, loc='upper right', title='Chromosomes')
    
    plt.xlim(0, cum_pos)
    plt.tight_layout()
    
    if output_dir:
        fig_file = os.path.join(output_dir, f"{patient_id}_manhattan_plot.png")
        plt.savefig(fig_file, dpi=300)
        plt.close()
    else:
        plt.show()

def visualize_vafs(data, by_sample=True, output_dir=None, patient_id=None):
    multiple_samples = 'SAMPLE' in data.columns and len(data['SAMPLE'].unique()) > 1
    
    if multiple_samples and by_sample:
        samples = data['SAMPLE'].unique()
        palette = sns.color_palette("tab10", n_colors=len(samples))
        sample_colors = dict(zip(samples, palette))
    
    # Save each plot separately
    
    # 1. VAF Distribution Histogram
    plt.figure(figsize=(8, 6))
    if multiple_samples and by_sample:
        for sample, color in sample_colors.items():
            sample_data = data[data['SAMPLE'] == sample]
            sns.histplot(sample_data['VAF'], bins=20, kde=True, 
                         label=sample, color=color, alpha=0.5)
        plt.legend(title='Sample')
    else:
        sns.histplot(data['VAF'], bins=20, kde=True)
    
    plt.xlabel('Variant Allele Fraction (VAF)')
    plt.ylabel('Number of Variants')
    plt.title('Distribution of Variant Allele Fractions')
    plt.tight_layout()
    
    if output_dir:
        fig_file = os.path.join(output_dir, f"{patient_id}_vaf_histogram.png")
        plt.savefig(fig_file, dpi=300)
        plt.close()
    else:
        plt.show()
    
    # 2. Scatter plot of VAF vs Read Depth
    plt.figure(figsize=(8, 6))
    if multiple_samples and by_sample:
        for sample, color in sample_colors.items():
            sample_data = data[data['SAMPLE'] == sample]
            plt.scatter(sample_data['TUMOR_DEPTH'], sample_data['VAF'], 
                       alpha=0.5, color=color, label=sample)
        plt.legend(title='Sample')
    else:
        plt.scatter(data['TUMOR_DEPTH'], data['VAF'], alpha=0.5)
    
    plt.xlabel('Tumor Read Depth')
    plt.ylabel('VAF')
    plt.title('VAF vs. Tumor Read Depth')
    plt.tight_layout()
    
    if output_dir:
        fig_file = os.path.join(output_dir, f"{patient_id}_depth_vs_vaf.png")
        plt.savefig(fig_file, dpi=300)
        plt.close()
    else:
        plt.show()
    
    # 3. VAF by chromosome (boxplot)
    plt.figure(figsize=(10, 6))
    
    chrom_counts = data['CHROM'].value_counts()
    top_chroms = chrom_counts.index[:10].tolist()
    chrom_data = data[data['CHROM'].isin(top_chroms)]
    
    if multiple_samples and by_sample:
        sns.boxplot(x='CHROM', y='VAF', hue='SAMPLE', data=chrom_data, palette=sample_colors)
        plt.legend(title='Sample', loc='upper right')
    else:
        sns.boxplot(x='CHROM', y='VAF', data=chrom_data)
    
    plt.xticks(rotation=45)
    plt.title('VAF Distribution by Chromosome')
    plt.tight_layout()
    
    if output_dir:
        fig_file = os.path.join(output_dir, f"{patient_id}_chrom_boxplot.png")
        plt.savefig(fig_file, dpi=300)
        plt.close()
    else:
        plt.show()
    
    # 4. Alt Count vs. Total Count scatter plot
    plt.figure(figsize=(8, 6))
    
    if multiple_samples and by_sample:
        for sample, color in sample_colors.items():
            sample_data = data[data['SAMPLE'] == sample]
            plt.scatter(sample_data['TOTAL_COUNT'], sample_data['ALT_COUNT'], 
                       alpha=0.7, color=color, label=sample)
        plt.legend(title='Sample')
    else:
        scatter = plt.scatter(data['TOTAL_COUNT'], data['ALT_COUNT'], 
                             c=data['VAF'], cmap='viridis', 
                             alpha=0.7)
        plt.colorbar(scatter, label='VAF')
    
    max_count = max(data['TOTAL_COUNT'])
    plt.plot([0, max_count], [0, max_count/2], 'r--', label='VAF = 0.5')
    
    plt.xlabel('Total Read Count')
    plt.ylabel('Alternate Allele Count')
    plt.title('Alternate vs. Total Read Count')
    plt.legend()
    plt.tight_layout()
    
    if output_dir:
        fig_file = os.path.join(output_dir, f"{patient_id}_alt_vs_total.png")
        plt.savefig(fig_file, dpi=300)
        plt.close()
    else:
        plt.show()
    
    # 5. Create VIOLIN plot for VAF distribution by sample
    if multiple_samples:
        plt.figure(figsize=(10, 6))
        
        # Use violin plot instead of boxplot
        sns.violinplot(x='SAMPLE', y='VAF', data=data, palette=sample_colors if by_sample else None,
                     inner='quartile')
        
        plt.xticks(rotation=45)
        plt.title('VAF Distribution by Sample (Violin Plot)')
        plt.tight_layout()
        
        if output_dir:
            fig_file = os.path.join(output_dir, f"{patient_id}_sample_violin.png")
            plt.savefig(fig_file, dpi=300)
            plt.close()
        else:
            plt.show()
    
    # 6. Create Manhattan plot
    create_manhattan_plot(data, by_sample=by_sample, output_dir=output_dir, patient_id=patient_id)

def generate_summary_stats(data, output_dir=None, patient_id=None):
    """
    Generate and save summary statistics for the VAF data
    This is a renamed function to avoid confusion with any other functions
    """
    # Overall stats
    overall_stats = {
        "Total variants": len(data),
        "Mean VAF": data['VAF'].mean(),
        "Median VAF": data['VAF'].median(),
        "Min VAF": data['VAF'].min(),
        "Max VAF": data['VAF'].max()
    }
    
    overall_df = pd.DataFrame([overall_stats])
    
    # Stats by sample if multiple samples
    if 'SAMPLE' in data.columns and len(data['SAMPLE'].unique()) > 1:
        sample_summary = data.groupby('SAMPLE').agg({
            'VAF': ['count', 'mean', 'median', 'min', 'max'],
            'TUMOR_DEPTH': ['mean', 'median']
        })
        sample_summary.columns = ['Variant Count', 'Mean VAF', 'Median VAF', 'Min VAF', 'Max VAF', 
                               'Mean Depth', 'Median Depth']
        sample_summary = sample_summary.round(4)
        
        # Chromosome distribution
        chrom_counts = data['CHROM'].value_counts().reset_index()
        chrom_counts.columns = ['Chromosome', 'Count']
        
        # VAF range distribution
        vaf_ranges = [
            (0.0, 0.1), (0.1, 0.2), (0.2, 0.3), (0.3, 0.4),
            (0.4, 0.5), (0.5, 0.6), (0.6, 1.0)
        ]
        
        vaf_range_data = []
        for vaf_min, vaf_max in vaf_ranges:
            count = ((data['VAF'] >= vaf_min) & (data['VAF'] < vaf_max)).sum()
            percent = (count / len(data)) * 100
            vaf_range_data.append({
                'Range': f"{vaf_min:.1f}-{vaf_max:.1f}",
                'Count': count,
                'Percentage': f"{percent:.1f}%"
            })
        
        vaf_range_df = pd.DataFrame(vaf_range_data)
        
        # Save to CSV files if output directory is provided
        if output_dir:
            overall_stats_file = os.path.join(output_dir, f"{patient_id}_overall_stats.csv")
            sample_summary_file = os.path.join(output_dir, f"{patient_id}_sample_summary.csv")
            chrom_file = os.path.join(output_dir, f"{patient_id}_chromosome_counts.csv")
            vaf_range_file = os.path.join(output_dir, f"{patient_id}_vaf_ranges.csv")
            
            overall_df.to_csv(overall_stats_file, index=False)
            sample_summary.to_csv(sample_summary_file)
            chrom_counts.to_csv(chrom_file, index=False)
            vaf_range_df.to_csv(vaf_range_file, index=False)
            
            print(f"Summary statistics saved to: {output_dir}")
        else:
            # Display in notebook
            print("Overall Statistics:")
            display(overall_df)
            
            print("\nSample Summary:")
            display(sample_summary)
            
            print("\nChromosome Distribution:")
            display(chrom_counts)
            
            print("\nVAF Range Distribution:")
            display(vaf_range_df)
    
    return {
        'overall': overall_df,
        'sample_summary': sample_summary if 'sample_summary' in locals() else None,
        'chrom_counts': chrom_counts if 'chrom_counts' in locals() else None,
        'vaf_ranges': vaf_range_df if 'vaf_range_df' in locals() else None
    }

def compare_samples_shared_variants(data, output_dir=None, patient_id=None):
    """
    Compare variants shared between samples
    """
    if 'SAMPLE' not in data.columns or len(data['SAMPLE'].unique()) <= 1:
        print("This function requires data from multiple samples")
        return
    
    data['VARIANT_ID'] = data['CHROM'] + '_' + data['POS'].astype(str) + '_' + data['REF'] + '_' + data['ALT']
    
    variant_counts = data['VARIANT_ID'].value_counts().reset_index()
    variant_counts.columns = ['VARIANT_ID', 'SAMPLE_COUNT']
    
    shared_data = pd.merge(data, variant_counts, on='VARIANT_ID')
    
    samples = data['SAMPLE'].unique()
    num_samples = len(samples)
    
    # Create summary dataframe for shared variants
    shared_summary = []
    for i in range(1, num_samples + 1):
        count = (shared_data['SAMPLE_COUNT'] == i).sum() // i  # Divide by i because each variant appears i times
        percent = (count / len(variant_counts)) * 100
        
        if i == 1:
            category = "Unique to one sample"
        elif i == num_samples:
            category = f"Shared across all {num_samples} samples"
        else:
            category = f"Shared across {i} samples"
            
        shared_summary.append({
            'Category': category,
            'Count': count,
            'Percentage': f"{percent:.1f}%"
        })
    
    shared_summary_df = pd.DataFrame(shared_summary)
    
    # Save or display
    if output_dir:
        summary_file = os.path.join(output_dir, f"{patient_id}_shared_variants_summary.csv")
        shared_summary_df.to_csv(summary_file, index=False)
    else:
        print("Shared Variants Summary:")
        display(shared_summary_df)
    
    if num_samples >= 2:
        shared_variants = variant_counts[variant_counts['SAMPLE_COUNT'] >= 2]['VARIANT_ID'].tolist()
        
        if shared_variants:
            shared_data = data[data['VARIANT_ID'].isin(shared_variants)]
            
            pivot_vaf = shared_data.pivot_table(
                index='VARIANT_ID', 
                columns='SAMPLE', 
                values='VAF', 
                aggfunc='mean'
            ).reset_index()
            
            if output_dir:
                pivot_file = os.path.join(output_dir, f"{patient_id}_shared_variants_vaf.csv")
                pivot_vaf.to_csv(pivot_file)
                print(f"Shared variants VAF comparison saved to: {pivot_file}")
            else:
                print("\nSample of shared variants VAF comparison:")
                display(pivot_vaf.head(10))
            
            if num_samples > 1:
                plt.figure(figsize=(15, 10))
                plot_idx = 1
                
                for i in range(num_samples):
                    for j in range(i+1, num_samples):
                        sample1 = samples[i]
                        sample2 = samples[j]
                        
                        pair_data = pivot_vaf[[sample1, sample2]].dropna()
                        
                        if len(pair_data) > 0:
                            corr = pair_data[sample1].corr(pair_data[sample2])
                            
                            plt.subplot(num_samples-1, num_samples-1, plot_idx)
                            plt.scatter(pair_data[sample1], pair_data[sample2], alpha=0.5)
                            plt.plot([0, 1], [0, 1], 'r--')  # Identity line
                            plt.xlabel(f'{sample1} VAF')
                            plt.ylabel(f'{sample2} VAF')
                            plt.title(f'Correlation: {corr:.3f}')
                            plt.axis('square')
                            plt.xlim(0, 1)
                            plt.ylim(0, 1)
                        
                        plot_idx += 1
                
                plt.tight_layout()
                
                if output_dir:
                    fig_file = os.path.join(output_dir, f"{patient_id}_vaf_correlations.png")
                    plt.savefig(fig_file, dpi=300)
                    plt.close()
                else:
                    plt.show()
        else:
            print("No variants are shared between samples")
    
    return shared_summary_df

def save_results(data, output_dir, base_filename):
    """
    Save the parsed data to TSV format
    """
    # Save to TSV (full dataset with all columns)
    tsv_path = os.path.join(output_dir, f"{base_filename}.tsv")
    data.to_csv(tsv_path, sep='\t', index=False)
    print(f"Data saved to {tsv_path}")
    
    # Generate and save summary statistics
    stats = generate_summary_stats(data, output_dir=output_dir, patient_id=base_filename.split('_')[0])
    
    # Save to Excel if more than one sample
    if 'SAMPLE' in data.columns and len(data['SAMPLE'].unique()) > 1:
        excel_path = os.path.join(output_dir, f"{base_filename}.xlsx")
        
        with pd.ExcelWriter(excel_path) as writer:
            data.to_excel(writer, sheet_name='All Variants', index=False)
            
            # Add samples to separate sheets
            for sample in data['SAMPLE'].unique():
                sample_data = data[data['SAMPLE'] == sample]
                sample_data.to_excel(writer, sheet_name=sample, index=False)
                
            # Add summary sheets
            if stats['overall'] is not None:
                stats['overall'].to_excel(writer, sheet_name='Overall_Stats', index=False)
            if stats['sample_summary'] is not None:
                stats['sample_summary'].to_excel(writer, sheet_name='Sample_Summary')
            if stats['vaf_ranges'] is not None:
                stats['vaf_ranges'].to_excel(writer, sheet_name='VAF_Ranges', index=False)
        
        print(f"Excel workbook with multiple sheets saved to {excel_path}")
    
    return {
        'full_path': tsv_path,
        'excel_path': excel_path if 'excel_path' in locals() else None
    }

def process_patient_vcfs(data_dir=DATA_DIR, results_dir=RESULTS_DIR, patient_id=None):
    """
    Process VCF files for a specific patient or all patients
    """
    # Find all VCF files
    all_vcf_files = find_vcf_files(data_dir)
    
    # Filter by patient if specified
    if patient_id:
        vcf_files = [f for f in all_vcf_files if f['patient_id'] == patient_id]
    else:
        vcf_files = all_vcf_files
    
    # Group by patient
    patient_vcfs = {}
    for vcf_file in vcf_files:
        patient = vcf_file['patient_id']
        if patient not in patient_vcfs:
            patient_vcfs[patient] = []
        patient_vcfs[patient].append(vcf_file)
    
    # Process each patient's VCF files
    patient_variants = {}
    
    for patient, files in patient_vcfs.items():
        print(f"Processing patient: {patient}")
        
        # Create results directory for this patient
        patient_results_dir = os.path.join(results_dir, patient)
        
        all_patient_variants = []
        
        # Process each sample for this patient
        for file_info in files:
            vcf_path = file_info['vcf_path']
            sample_id = file_info['sample_id']
            
            print(f"  Processing sample: {sample_id}, file: {os.path.basename(vcf_path)}")
            
            try:
                variants_df = parse_strelka2_vcf(vcf_path, sample_name=sample_id, patient_id=patient)
                all_patient_variants.append(variants_df)
                print(f"    Found {len(variants_df)} variants in sample {sample_id}")
            except Exception as e:
                print(f"    Error processing {vcf_path}: {e}")
        
        # Combine all variants for this patient
        if all_patient_variants:
            combined_df = pd.concat(all_patient_variants, ignore_index=True)
            print(f"  Total variants for patient {patient}: {len(combined_df)}")
            
            # Save the variants
            save_results(combined_df, patient_results_dir, f"{patient}_variants")
            
            # Generate and save visualizations
            print(f"  Generating visualizations for patient {patient}...")
            visualize_vafs(combined_df, by_sample=True, output_dir=patient_results_dir, patient_id=patient)
            
            # Compare samples if there are multiple
            if 'SAMPLE' in combined_df.columns and len(combined_df['SAMPLE'].unique()) > 1:
                print(f"  Comparing samples for patient {patient}...")
                compare_samples_shared_variants(combined_df, output_dir=patient_results_dir, patient_id=patient)
            
            # Save the data for return
            patient_variants[patient] = combined_df
        else:
            print(f"  No variants found for patient {patient}")
    
    return patient_variants

# For use in Jupyter Notebook - run this cell to process all patients
if __name__ == "__main__":
    # Process all patients or specific patient
    # To process a specific patient, use: process_patient_vcfs(patient_id='G12')
    all_patient_variants = process_patient_vcfs()
    
    print(f"\nProcessed {len(all_patient_variants)} patients:")
    for patient, variants in all_patient_variants.items():
        sample_count = len(variants['SAMPLE'].unique())
        print(f"  - {patient}: {len(variants)} variants across {sample_count} samples")

Processing patient: p191
  Processing sample: 41126, file: somatic.XY.snvs.pass.vcf
    Found 434 variants in sample 41126
  Processing sample: 41133, file: somatic.XY.snvs.pass.vcf
    Found 440 variants in sample 41133
  Processing sample: 41114, file: somatic.XY.snvs.pass.vcf
    Found 398 variants in sample 41114
  Processing sample: 42407, file: somatic.XY.snvs.pass.vcf
    Found 313 variants in sample 42407
  Total variants for patient p191: 1585
Data saved to results/p191/p191_variants.tsv
Summary statistics saved to: results/p191
Excel workbook with multiple sheets saved to results/p191/p191_variants.xlsx
  Generating visualizations for patient p191...


/var/folders/ct/zks035_n5vn9xb6d7n4xk5200000gn/T/ipykernel_21952/839645181.py:308: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='SAMPLE', y='VAF', data=data, palette=sample_colors if by_sample else None,


  Comparing samples for patient p191...
Shared variants VAF comparison saved to: results/p191/p191_shared_variants_vaf.csv
Processing patient: P163
  Processing sample: 375363, file: somatic.XX.snvs.pass.vcf
    Found 576 variants in sample 375363
  Processing sample: 376223, file: somatic.XX.snvs.pass.vcf
    Found 671 variants in sample 376223
  Processing sample: 376213, file: somatic.XX.snvs.pass.vcf
    Found 651 variants in sample 376213
  Processing sample: 376137, file: somatic.XX.snvs.pass.vcf
    Found 467 variants in sample 376137
  Processing sample: 376121, file: somatic.XX.snvs.pass.vcf
    Found 888 variants in sample 376121
  Processing sample: 375380, file: somatic.XX.snvs.pass.vcf
    Found 597 variants in sample 375380
  Total variants for patient P163: 3850
Data saved to results/P163/P163_variants.tsv
Summary statistics saved to: results/P163
Excel workbook with multiple sheets saved to results/P163/P163_variants.xlsx
  Generating visualizations for patient P163...


/var/folders/ct/zks035_n5vn9xb6d7n4xk5200000gn/T/ipykernel_21952/839645181.py:308: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='SAMPLE', y='VAF', data=data, palette=sample_colors if by_sample else None,


  Comparing samples for patient P163...
Shared variants VAF comparison saved to: results/P163/P163_shared_variants_vaf.csv
Processing patient: P190
  Processing sample: 359388, file: somatic.XX.snvs.pass.vcf
    Found 802 variants in sample 359388
  Processing sample: 359227, file: somatic.XX.snvs.pass.vcf
    Found 891 variants in sample 359227
  Processing sample: 359287, file: somatic.XX.snvs.pass.vcf
    Found 611 variants in sample 359287
  Processing sample: 359261, file: somatic.XX.snvs.pass.vcf
    Found 581 variants in sample 359261
  Processing sample: 359313, file: somatic.XX.snvs.pass.vcf
    Found 598 variants in sample 359313
  Total variants for patient P190: 3483
Data saved to results/P190/P190_variants.tsv
Summary statistics saved to: results/P190
Excel workbook with multiple sheets saved to results/P190/P190_variants.xlsx
  Generating visualizations for patient P190...


...

  Comparing samples for patient G45...
Shared variants VAF comparison saved to: results/G45/G45_shared_variants_vaf.csv
Processing patient: G43
  Processing sample: R, file: somatic.XY.snvs.pass.vcf
    Found 1807 variants in sample R
  Processing sample: I, file: somatic.XY.snvs.pass.vcf
    Found 881 variants in sample I
  Processing sample: T, file: somatic.XY.snvs.pass.vcf
    Found 781 variants in sample T
  Processing sample: C, file: somatic.XY.snvs.pass.vcf
    Found 831 variants in sample C
  Processing sample: P, file: somatic.XY.snvs.pass.vcf
    Found 548 variants in sample P
  Total variants for patient G43: 4848
Data saved to results/G43/G43_variants.tsv
Summary statistics saved to: results/G43
Excel workbook with multiple sheets saved to results/G43/G43_variants.xlsx
  Generating visualizations for patient G43...


/var/folders/ct/zks035_n5vn9xb6d7n4xk5200000gn/T/ipykernel_21952/839645181.py:308: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='SAMPLE', y='VAF', data=data, palette=sample_colors if by_sample else None,


  Comparing samples for patient G43...
Shared variants VAF comparison saved to: results/G43/G43_shared_variants_vaf.csv
Processing patient: p129
  Processing sample: 376176, file: somatic.XX.snvs.pass.vcf
    Found 589 variants in sample 376176
  Processing sample: 376050, file: somatic.XX.snvs.pass.vcf
    Found 458 variants in sample 376050
  Processing sample: 375688, file: somatic.XX.snvs.pass.vcf
    Found 630 variants in sample 375688
  Processing sample: 376304, file: somatic.XX.snvs.pass.vcf
    Found 602 variants in sample 376304
  Processing sample: 376165, file: somatic.XX.snvs.pass.vcf
    Found 543 variants in sample 376165
  Processing sample: 375970, file: somatic.XX.snvs.pass.vcf
    Found 448 variants in sample 375970
  Processing sample: 376085, file: somatic.XX.snvs.pass.vcf
    Found 506 variants in sample 376085
  Processing sample: 375973, file: somatic.XX.snvs.pass.vcf
    Found 649 variants in sample 375973
  Processing sample: 375918, file: somatic.XX.snvs.pass

/var/folders/ct/zks035_n5vn9xb6d7n4xk5200000gn/T/ipykernel_21952/839645181.py:308: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='SAMPLE', y='VAF', data=data, palette=sample_colors if by_sample else None,


  Comparing samples for patient p129...
Shared variants VAF comparison saved to: results/p129/p129_shared_variants_vaf.csv

Processed 28 patients:
  - p191: 1585 variants across 4 samples
  - P163: 3850 variants across 6 samples
  - P190: 3483 variants across 5 samples
  - p152: 4215 variants across 7 samples
  - M36: 5331 variants across 7 samples
  - G53: 7620 variants across 9 samples
  - G39: 3851 variants across 6 samples
  - G97: 32021 variants across 8 samples
  - G52: 5098 variants across 10 samples
  - G99: 45509 variants across 4 samples
  - G77: 8844 variants across 8 samples
  - G12: 8161 variants across 4 samples
  - M24: 3305 variants across 4 samples
  - G23: 5157 variants across 6 samples
  - G40: 4941 variants across 9 samples
  - M78: 4871 variants across 5 samples
  - G85: 8416 variants across 7 samples
  - p182: 1905 variants across 4 samples
  - P158: 2854 variants across 5 samples
  - G5: 2967 variants across 6 samples
  - M95: 5495 variants across 6 samples
  - G

## PyClone-VI

### Convert the VAF data into the format expected by PyClone-VI

In [ ]:
# Standalone VCF to PyClone-VI Converter
# This code reads variant data from TSV files in the results directory
# and saves PyClone input files in the same patient directories

import pandas as pd
import os
import glob

def get_patient_variant_files(results_dir="results"):
    """
    Find all patient variant TSV files in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        
    Returns:
        dict: Dictionary mapping patient IDs to their variant file paths
    """
    patient_files = {}
    
    # Look for patient directories in the results directory
    patient_dirs = glob.glob(os.path.join(results_dir, "*"))
    
    for patient_dir in patient_dirs:
        if os.path.isdir(patient_dir):
            # Extract patient ID from directory name
            patient_id = os.path.basename(patient_dir)
            
            # Look for variant TSV file with patient ID prefix
            variant_file_pattern = os.path.join(patient_dir, f"{patient_id}_variants.tsv")
            variant_files = glob.glob(variant_file_pattern)
            
            if variant_files:
                patient_files[patient_id] = variant_files[0]
                print(f"Found variant file for patient {patient_id}: {variant_files[0]}")
    
    if not patient_files:
        print(f"No patient variant files found in {results_dir}")
    
    return patient_files

def process_patient_variants(patient_id, variant_file, output_dir=None, 
                            default_tumor_content=0.5,
                            min_coverage=20, min_vaf=0.05):
    """
    Process a single patient's variants and convert to PyClone format
    
    Args:
        patient_id (str): Patient identifier
        variant_file (str): Path to patient's variant TSV file
        output_dir (str): Directory to save output (if None, save in same directory as input)
        default_tumor_content (float): Default tumor purity estimate (0-1)
        min_coverage (int): Minimum read coverage to include a variant
        min_vaf (float): Minimum VAF to include a variant
        
    Returns:
        pd.DataFrame: PyClone formatted data for the patient
    """
    # Determine output directory
    if output_dir is None:
        output_dir = os.path.dirname(variant_file)
    
    print(f"Processing patient {patient_id}...")
    
    try:
        # Read variant file
        variants_df = pd.read_csv(variant_file, sep='\t')
        print(f"  Loaded {len(variants_df)} variants")
        
        # Make sure we have the required columns
        required_columns = ['CHROM', 'POS', 'REF', 'ALT', 'REF_COUNT', 'ALT_COUNT', 'TOTAL_COUNT', 'VAF', 'SAMPLE']
        missing_columns = [col for col in required_columns if col not in variants_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns in variant file: {', '.join(missing_columns)}")
        
        # Filter variants by coverage and VAF
        filtered_variants = variants_df[
            (variants_df['TOTAL_COUNT'] >= min_coverage) & 
            (variants_df['VAF'] >= min_vaf)
        ].copy()
        
        print(f"  Filtered from {len(variants_df)} to {len(filtered_variants)} variants")
        print(f"    Minimum coverage: {min_coverage}")
        print(f"    Minimum VAF: {min_vaf}")
        
        # Get unique samples for this patient
        samples = filtered_variants['SAMPLE'].unique()
        print(f"  Found {len(samples)} samples: {', '.join(samples)}")
        
        # Process each sample
        sample_pyclone_dfs = {}
        
        for sample in samples:
            sample_data = filtered_variants[filtered_variants['SAMPLE'] == sample]
            print(f"  Processing sample {sample}: {len(sample_data)} variants")
            
            # Convert to PyClone format
            pyclone_data = []
            for _, row in sample_data.iterrows():
                # Create mutation_id from chromosome and position
                mutation_id = f"{row['CHROM']}:{row['POS']}:{row['REF']}>{row['ALT']}"
                
                # Set default copy number values (assuming diploid)
                normal_cn = 2  # Assuming autosomal
                major_cn = 1   # Default major copy number
                minor_cn = 1   # Default minor copy number
                
                # Adjust for sex chromosomes
                if row['CHROM'] in ['chrX', 'X', 'chrY', 'Y']:
                    if row['CHROM'] in ['chrX', 'X']:
                        # Set X chromosome values (assuming female)
                        pass  # Using defaults
                    elif row['CHROM'] in ['chrY', 'Y']:
                        # Set Y chromosome values (assuming male)
                        normal_cn = 1
                        major_cn = 1
                        minor_cn = 0
                
                # Add to PyClone data
                pyclone_data.append({
                    'mutation_id': mutation_id,
                    'sample_id': sample,
                    'ref_counts': row['REF_COUNT'],
                    'alt_counts': row['ALT_COUNT'],
                    'normal_cn': normal_cn,
                    'major_cn': major_cn,
                    'minor_cn': minor_cn,
                    'tumor_content': default_tumor_content
                })
            
            # Create PyClone DataFrame for this sample
            sample_pyclone_df = pd.DataFrame(pyclone_data)
            sample_pyclone_dfs[sample] = sample_pyclone_df
        
        # Combine all samples for this patient into a single PyClone file
        combined_pyclone_df = pd.concat(sample_pyclone_dfs.values(), ignore_index=True)
        
        # Save combined file to patient's directory
        output_file = os.path.join(output_dir, f"{patient_id}_pyclone_input.tsv")
        combined_pyclone_df.to_csv(output_file, sep='\t', index=False)
        print(f"  Saved combined PyClone input for patient {patient_id} to {output_file}")
        
        return combined_pyclone_df
        
    except Exception as e:
        print(f"  Error processing patient {patient_id}: {e}")
        return None

# Main execution block
if __name__ == "__main__":
    # Set parameters
    results_directory = "results"  # Directory containing patient subdirectories
    tumor_purity = 1.0            # Estimated tumor purity (0-1)
    min_read_coverage = 1         # Minimum total reads required
    min_vaf_threshold = 0.0       # Minimum VAF required (0%)
    
    # Get all patient variant files
    print(f"Finding patient variant files in {results_directory}")
    patient_files = get_patient_variant_files(results_dir=results_directory)
    
    if not patient_files:
        print("No patient variant files found. Exiting.")
        exit(1)
    
    # Process each patient independently
    pyclone_results = {}
    for patient_id, variant_file in patient_files.items():
        pyclone_df = process_patient_variants(
            patient_id,
            variant_file,
            default_tumor_content=tumor_purity,
            min_coverage=min_read_coverage,
            min_vaf=min_vaf_threshold
        )
        
        if pyclone_df is not None:
            pyclone_results[patient_id] = pyclone_df
    
    # Display summary
    print("\nPyClone conversion complete:")
    for patient, data in pyclone_results.items():
        print(f"  Patient {patient}: {len(data)} variants processed across {len(data['sample_id'].unique())} samples")
        
    # Display the first few rows of the first patient's PyClone input
    if pyclone_results:
        first_patient = list(pyclone_results.keys())[0]
        print(f"\nFirst few rows of PyClone input for patient {first_patient}:")
        print(pyclone_results[first_patient].head())

Finding patient variant files in results
Found variant file for patient p191: results/p191/p191_variants.tsv
Found variant file for patient P163: results/P163/P163_variants.tsv
Found variant file for patient P190: results/P190/P190_variants.tsv
Found variant file for patient p152: results/p152/p152_variants.tsv
Found variant file for patient M36: results/M36/M36_variants.tsv
Found variant file for patient G53: results/G53/G53_variants.tsv
Found variant file for patient G39: results/G39/G39_variants.tsv
Found variant file for patient G97: results/G97/G97_variants.tsv
Found variant file for patient G52: results/G52/G52_variants.tsv
Found variant file for patient G99: results/G99/G99_variants.tsv
Found variant file for patient G77: results/G77/G77_variants.tsv
Found variant file for patient G12: results/G12/G12_variants.tsv
Found variant file for patient M24: results/M24/M24_variants.tsv
Found variant file for patient G23: results/G23/G23_variants.tsv
Found variant file for patient G40: r

### add default for missing

In [429]:
# Standalone VCF to PyClone-VI Converter
# This code reads variant data from TSV files in the results directory
# and saves PyClone input files in the same patient directories

import pandas as pd
import os
import glob

def get_patient_variant_files(results_dir="results"):
    """
    Find all patient variant TSV files in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        
    Returns:
        dict: Dictionary mapping patient IDs to their variant file paths
    """
    patient_files = {}
    
    # Look for patient directories in the results directory
    patient_dirs = glob.glob(os.path.join(results_dir, "*"))
    
    for patient_dir in patient_dirs:
        if os.path.isdir(patient_dir):
            # Extract patient ID from directory name
            patient_id = os.path.basename(patient_dir)
            
            # Look for variant TSV file with patient ID prefix
            variant_file_pattern = os.path.join(patient_dir, f"{patient_id}_variants.tsv")
            variant_files = glob.glob(variant_file_pattern)
            
            if variant_files:
                patient_files[patient_id] = variant_files[0]
                print(f"Found variant file for patient {patient_id}: {variant_files[0]}")
    
    if not patient_files:
        print(f"No patient variant files found in {results_dir}")
    
    return patient_files

def process_patient_variants(patient_id, variant_file, output_dir=None, 
                            default_tumor_content=0.5,
                            min_coverage=20, min_vaf=0.05):
    """
    Process a single patient's variants and convert to PyClone format
    
    Args:
        patient_id (str): Patient identifier
        variant_file (str): Path to patient's variant TSV file
        output_dir (str): Directory to save output (if None, save in same directory as input)
        default_tumor_content (float): Default tumor purity estimate (0-1)
        min_coverage (int): Minimum read coverage to include a variant
        min_vaf (float): Minimum VAF to include a variant
        
    Returns:
        pd.DataFrame: PyClone formatted data for the patient
    """
    # Determine output directory
    if output_dir is None:
        output_dir = os.path.dirname(variant_file)
    
    print(f"Processing patient {patient_id}...")
    
    try:
        # Read variant file
        variants_df = pd.read_csv(variant_file, sep='\t')
        print(f"  Loaded {len(variants_df)} variants")
        
        # Make sure we have the required columns
        required_columns = ['CHROM', 'POS', 'REF', 'ALT', 'REF_COUNT', 'ALT_COUNT', 'TOTAL_COUNT', 'VAF', 'SAMPLE']
        missing_columns = [col for col in required_columns if col not in variants_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns in variant file: {', '.join(missing_columns)}")
        
        # Filter variants by coverage and VAF
        filtered_variants = variants_df[
            (variants_df['TOTAL_COUNT'] >= min_coverage) & 
            (variants_df['VAF'] >= min_vaf)
        ].copy()
        
        print(f"  Filtered from {len(variants_df)} to {len(filtered_variants)} variants")
        print(f"    Minimum coverage: {min_coverage}")
        print(f"    Minimum VAF: {min_vaf}")
        
        # Get unique samples for this patient
        samples = filtered_variants['SAMPLE'].unique()
        print(f"  Found {len(samples)} samples: {', '.join(samples)}")
        
        # Create a comprehensive list of all mutations across all samples
        all_mutations = set()
        for _, row in filtered_variants.iterrows():
            mutation_id = f"{row['CHROM']}:{row['POS']}:{row['REF']}>{row['ALT']}"
            all_mutations.add(mutation_id)
        
        print(f"  Found {len(all_mutations)} unique mutations across all samples")
        
        # Process each sample
        sample_pyclone_dfs = {}
        
        for sample in samples:
            sample_data = filtered_variants[filtered_variants['SAMPLE'] == sample]
            print(f"  Processing sample {sample}: {len(sample_data)} variants")
            
            # Create a set of mutations present in this sample
            sample_mutations = set()
            for _, row in sample_data.iterrows():
                mutation_id = f"{row['CHROM']}:{row['POS']}:{row['REF']}>{row['ALT']}"
                sample_mutations.add(mutation_id)
            
            # Convert to PyClone format
            pyclone_data = []
            
            # Process mutations present in this sample
            for _, row in sample_data.iterrows():
                mutation_id = f"{row['CHROM']}:{row['POS']}:{row['REF']}>{row['ALT']}"
                
                # Set default copy number values (assuming diploid)
                normal_cn = 2  # Assuming autosomal
                major_cn = 1   # Default major copy number
                minor_cn = 1   # Default minor copy number
                
                # Adjust for sex chromosomes
                if row['CHROM'] in ['chrX', 'X', 'chrY', 'Y']:
                    if row['CHROM'] in ['chrX', 'X']:
                        # Set X chromosome values (assuming female)
                        pass  # Using defaults
                    elif row['CHROM'] in ['chrY', 'Y']:
                        # Set Y chromosome values (assuming male)
                        normal_cn = 1
                        major_cn = 1
                        minor_cn = 0
                
                # Add to PyClone data
                pyclone_data.append({
                    'mutation_id': mutation_id,
                    'sample_id': sample,
                    'ref_counts': row['REF_COUNT'],
                    'alt_counts': row['ALT_COUNT'],
                    'normal_cn': normal_cn,
                    'major_cn': major_cn,
                    'minor_cn': minor_cn,
                    'tumor_content': default_tumor_content
                })
            
            # Add missing mutations with default values (100 ref, 0 alt)
            missing_mutations = all_mutations - sample_mutations
            for mutation_id in missing_mutations:
                # Parse chromosome from mutation ID
                chrom = mutation_id.split(':')[0]
                
                # Set default copy number values (assuming diploid)
                normal_cn = 2  # Assuming autosomal
                major_cn = 1   # Default major copy number
                minor_cn = 1   # Default minor copy number
                
                # Adjust for sex chromosomes
                if chrom in ['chrX', 'X', 'chrY', 'Y']:
                    if chrom in ['chrX', 'X']:
                        # Set X chromosome values (assuming female)
                        pass  # Using defaults
                    elif chrom in ['chrY', 'Y']:
                        # Set Y chromosome values (assuming male)
                        normal_cn = 1
                        major_cn = 1
                        minor_cn = 0
                
                # Add to PyClone data with default counts
                pyclone_data.append({
                    'mutation_id': mutation_id,
                    'sample_id': sample,
                    'ref_counts': 100,  # Default reference count for missing mutations
                    'alt_counts': 0,    # Default alternate count for missing mutations
                    'normal_cn': normal_cn,
                    'major_cn': major_cn,
                    'minor_cn': minor_cn,
                    'tumor_content': default_tumor_content
                })
            
            # Create PyClone DataFrame for this sample
            sample_pyclone_df = pd.DataFrame(pyclone_data)
            sample_pyclone_dfs[sample] = sample_pyclone_df
            
            print(f"    Added {len(missing_mutations)} missing mutations with default values")
        
        # Combine all samples for this patient into a single PyClone file
        combined_pyclone_df = pd.concat(sample_pyclone_dfs.values(), ignore_index=True)
        
        # Save combined file to patient's directory
        output_file = os.path.join(output_dir, f"{patient_id}_pyclone_input.tsv")
        combined_pyclone_df.to_csv(output_file, sep='\t', index=False)
        print(f"  Saved combined PyClone input for patient {patient_id} to {output_file}")
        
        return combined_pyclone_df
        
    except Exception as e:
        print(f"  Error processing patient {patient_id}: {e}")
        return None

# Main execution block
if __name__ == "__main__":
    # Set parameters
    results_directory = "results"  # Directory containing patient subdirectories
    tumor_purity = 1.0            # Estimated tumor purity (0-1)
    min_read_coverage = 50        # Minimum total reads required
    min_vaf_threshold = 0.01     
    
    # Get all patient variant files
    print(f"Finding patient variant files in {results_directory}")
    patient_files = get_patient_variant_files(results_dir=results_directory)
    
    if not patient_files:
        print("No patient variant files found. Exiting.")
        exit(1)
    
    # Process each patient independently
    pyclone_results = {}
    for patient_id, variant_file in patient_files.items():
        pyclone_df = process_patient_variants(
            patient_id,
            variant_file,
            default_tumor_content=tumor_purity,
            min_coverage=min_read_coverage,
            min_vaf=min_vaf_threshold
        )
        
        if pyclone_df is not None:
            pyclone_results[patient_id] = pyclone_df
    
    # Display summary
    print("\nPyClone conversion complete:")
    for patient, data in pyclone_results.items():
        print(f"  Patient {patient}: {len(data)} variants processed across {len(data['sample_id'].unique())} samples")
        
    # # Display the first few rows of the first patient's PyClone input
    # if pyclone_results:
    #     first_patient = list(pyclone_results.keys())[0]
    #     print(f"\nFirst few rows of PyClone input for patient {first_patient}:")
    #     print(pyclone_results[first_patient].head())

Finding patient variant files in results
Found variant file for patient p191: results/p191/p191_variants.tsv
Found variant file for patient P163: results/P163/P163_variants.tsv
Found variant file for patient P190: results/P190/P190_variants.tsv
Found variant file for patient p152: results/p152/p152_variants.tsv
Found variant file for patient M36: results/M36/M36_variants.tsv
Found variant file for patient G53: results/G53/G53_variants.tsv
Found variant file for patient G39: results/G39/G39_variants.tsv
Found variant file for patient G97: results/G97/G97_variants.tsv
Found variant file for patient G52: results/G52/G52_variants.tsv
Found variant file for patient G99: results/G99/G99_variants.tsv
Found variant file for patient G77: results/G77/G77_variants.tsv
Found variant file for patient G12: results/G12/G12_variants.tsv
Found variant file for patient M24: results/M24/M24_variants.tsv
Found variant file for patient G23: results/G23/G23_variants.tsv
Found variant file for patient G40: r

### Run the tool

In [20]:
# !pyclone-vi fit -i pyclone_input/combined_pyclone_input.tsv -o pyclone_results.h5 -c 40 -d beta-binomial -r 10

In [19]:
!pyclone-vi write-results-file -i pyclone_results.h5 -o pyclone_results.tsv

/Users/nick/Library/Python/3.9/lib/python/site-packages/pyclone_vi/post_process.py:64: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt(var)


## CloneFinder

### Convert the VAF data into the format expected by CloneFinder

In [22]:
# VCF to CloneFinder Converter - Final Version
# This script reads variant data from TSV files in the results directory
# and saves CloneFinder input files in the same patient directories

import pandas as pd
import os
import glob
import numpy as np

def get_patient_variant_files(results_dir="results"):
    """
    Find all patient variant TSV files in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        
    Returns:
        dict: Dictionary mapping patient IDs to their variant file paths
    """
    patient_files = {}
    
    # Look for patient directories in the results directory
    patient_dirs = glob.glob(os.path.join(results_dir, "*"))
    
    for patient_dir in patient_dirs:
        if os.path.isdir(patient_dir):
            # Extract patient ID from directory name
            patient_id = os.path.basename(patient_dir)
            
            # Look for variant TSV file with patient ID prefix
            variant_file_pattern = os.path.join(patient_dir, f"{patient_id}_variants.tsv")
            variant_files = glob.glob(variant_file_pattern)
            
            if variant_files:
                patient_files[patient_id] = variant_files[0]
                print(f"Found variant file for patient {patient_id}: {variant_files[0]}")
    
    if not patient_files:
        print(f"No patient variant files found in {results_dir}")
    
    return patient_files

def process_patient_variants_for_clonefinder(patient_id, variant_file, 
                                           min_coverage=20, min_vaf=0.05,
                                           use_simple_ids=False, 
                                           require_all_samples=False,
                                           missing_ref_count=100):
    """
    Process a single patient's variants and convert to CloneFinder format
    
    Args:
        patient_id (str): Patient identifier
        variant_file (str): Path to patient's variant TSV file
        min_coverage (int): Minimum read coverage to include a variant
        min_vaf (float): Minimum VAF to include a variant
        use_simple_ids (bool): If True, generate simple SNV IDs (S1, S2, etc.)
        require_all_samples (bool): If True, only include variants found in all samples
        missing_ref_count (int): Reference count to use for variants not found in a sample
        
    Returns:
        pd.DataFrame: CloneFinder formatted data for the patient
    """
    output_dir = os.path.dirname(variant_file)
    output_file = os.path.join(output_dir, f"{patient_id}_clonefinder_input.txt")
    
    print(f"Processing patient {patient_id}...")
    
    try:
        # Read variant file
        variants_df = pd.read_csv(variant_file, sep='\t')
        print(f"  Loaded {len(variants_df)} variants")
        
        # Check for required columns
        required_cols = ['CHROM', 'POS', 'REF', 'ALT', 'REF_COUNT', 'ALT_COUNT', 'SAMPLE']
        missing_columns = [col for col in required_cols if col not in variants_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns in variant file: {', '.join(missing_columns)}")
        
        # Ensure all string columns are strings (not integers/numerics)
        variants_df['CHROM'] = variants_df['CHROM'].astype(str)
        variants_df['POS'] = variants_df['POS'].astype(str)
        variants_df['REF'] = variants_df['REF'].astype(str)
        variants_df['ALT'] = variants_df['ALT'].astype(str)
        variants_df['SAMPLE'] = variants_df['SAMPLE'].astype(str)
        
        # Ensure numeric columns are numeric
        variants_df['REF_COUNT'] = pd.to_numeric(variants_df['REF_COUNT'])
        variants_df['ALT_COUNT'] = pd.to_numeric(variants_df['ALT_COUNT'])
        
        # Filter variants by coverage and VAF
        if 'TOTAL_COUNT' in variants_df.columns:
            total_count = pd.to_numeric(variants_df['TOTAL_COUNT'])
        else:
            total_count = variants_df['REF_COUNT'] + variants_df['ALT_COUNT']
            
        if 'VAF' in variants_df.columns:
            vaf = pd.to_numeric(variants_df['VAF'])
        else:
            vaf = variants_df['ALT_COUNT'] / total_count.replace(0, 1)  # Avoid division by zero
        
        filtered_variants = variants_df[
            (total_count >= min_coverage) & 
            (vaf >= min_vaf)
        ].copy()
        
        print(f"  Filtered from {len(variants_df)} to {len(filtered_variants)} variants")
        print(f"    Minimum coverage: {min_coverage}")
        print(f"    Minimum VAF: {min_vaf}")
        
        if len(filtered_variants) == 0:
            raise ValueError("No variants remain after filtering. Consider lowering thresholds.")
        
        # Get list of unique samples for this patient
        samples = sorted(filtered_variants['SAMPLE'].unique())
        print(f"  Found {len(samples)} samples for patient {patient_id}: {', '.join(samples)}")
        
        # Create mutation IDs
        if use_simple_ids:
            # Create simple sequential IDs (S1, S2, etc.)
            unique_variants = filtered_variants.drop_duplicates(['CHROM', 'POS', 'REF', 'ALT'])
            id_map = {
                (str(row['CHROM']), str(row['POS']), str(row['REF']), str(row['ALT'])): f"S{i+1}" 
                for i, (_, row) in enumerate(unique_variants.iterrows())
            }
            filtered_variants['SNVID'] = filtered_variants.apply(
                lambda row: id_map[(str(row['CHROM']), str(row['POS']), str(row['REF']), str(row['ALT']))], axis=1
            )
            snv_id_column = 'SNVID'
        else:
            print("  Creating mutation IDs based on genomic coordinates")
            filtered_variants['SNVID'] = filtered_variants.apply(
                lambda row: f"{str(row['CHROM'])}:{str(row['POS'])}:{str(row['REF'])}>{str(row['ALT'])}", axis=1
            )
            snv_id_column = 'SNVID'
        
        # Find variants across samples based on setting
        if len(samples) > 1:
            variant_counts = filtered_variants.groupby(snv_id_column)['SAMPLE'].nunique()
            
            if require_all_samples:
                # Only keep variants present in all samples
                common_variants = variant_counts[variant_counts == len(samples)].index.tolist()
                
                if len(common_variants) == 0:
                    print(f"  Warning: No variants found in all samples for patient {patient_id}. Consider setting require_all_samples=False")
                    if len(variant_counts) > 0:
                        # Find variants in the maximum number of samples
                        max_samples = variant_counts.max()
                        common_variants = variant_counts[variant_counts == max_samples].index.tolist()
                        print(f"  Including {len(common_variants)} variants found in {max_samples}/{len(samples)} samples")
                else:
                    print(f"  Found {len(common_variants)} variants present in all {len(samples)} samples")
            else:
                # Keep all variants
                common_variants = filtered_variants[snv_id_column].unique()
                print(f"  Including all {len(common_variants)} variants (may not be present in all samples)")
                
            filtered_variants = filtered_variants[filtered_variants[snv_id_column].isin(common_variants)]
        
        # Create a reference for the alleles
        variant_alleles = {}
        for _, row in filtered_variants.iterrows():
            var_id = row[snv_id_column]
            if var_id not in variant_alleles:
                variant_alleles[var_id] = {
                    'REF': str(row['REF']), 
                    'ALT': str(row['ALT'])
                }
        
        # Create a complete sample/variant matrix with appropriate values for missing variants
        all_variant_ids = filtered_variants[snv_id_column].unique()
        all_combinations = []
        
        for var_id in all_variant_ids:
            for sample in samples:
                sample_var_data = filtered_variants[
                    (filtered_variants[snv_id_column] == var_id) & 
                    (filtered_variants['SAMPLE'] == sample)
                ]
                
                if len(sample_var_data) > 0:
                    # Variant exists in this sample
                    ref_count = int(sample_var_data.iloc[0]['REF_COUNT'])
                    alt_count = int(sample_var_data.iloc[0]['ALT_COUNT'])
                else:
                    # Variant doesn't exist in this sample
                    # Use placeholder value for reference count and 0 for alt count
                    ref_count = missing_ref_count
                    alt_count = 0
                
                all_combinations.append({
                    snv_id_column: var_id,
                    'SAMPLE': sample,
                    'REF_COUNT': ref_count,
                    'ALT_COUNT': alt_count
                })
        
        # Create a new DataFrame with all combinations
        complete_variants = pd.DataFrame(all_combinations)
        
        # Create pivot tables for the reference and alternate counts
        pivot_ref = complete_variants.pivot_table(
            index=snv_id_column, 
            columns='SAMPLE', 
            values='REF_COUNT', 
            aggfunc='first'
        ).reset_index()
        
        pivot_alt = complete_variants.pivot_table(
            index=snv_id_column, 
            columns='SAMPLE', 
            values='ALT_COUNT', 
            aggfunc='first'
        )
        
        # Rename columns to match CloneFinder format
        pivot_ref.columns.name = None
        for sample in samples:
            pivot_ref.rename(columns={sample: f"{sample}:ref"}, inplace=True)
        
        pivot_alt.columns.name = None
        for sample in samples:
            pivot_alt.rename(columns={sample: f"{sample}:alt"}, inplace=True)
        
        # Merge the pivot tables
        clonefinder_data = pivot_ref.copy()
        
        # Add Wild and Mut columns (reference and alternate alleles)
        clonefinder_data['Wild'] = clonefinder_data[snv_id_column].map(
            {var_id: alleles['REF'] for var_id, alleles in variant_alleles.items()}
        )
        clonefinder_data['Mut'] = clonefinder_data[snv_id_column].map(
            {var_id: alleles['ALT'] for var_id, alleles in variant_alleles.items()}
        )
        
        # Reorder columns to match expected format
        cols = [snv_id_column, 'Wild', 'Mut']
        
        # Add reference count columns
        for sample in samples:
            cols.append(f"{sample}:ref")
        
        # Add alternate count columns
        for sample in samples:
            cols.append(f"{sample}:alt")
            clonefinder_data[f"{sample}:alt"] = pivot_alt[f"{sample}:alt"].values
        
        # Select and order columns
        clonefinder_data = clonefinder_data[cols]
        
        # Save to file
        clonefinder_data.to_csv(output_file, sep='\t', index=False)
        print(f"  CloneFinder input saved to {output_file}")
        
        return clonefinder_data
        
    except Exception as e:
        print(f"  Error processing patient {patient_id}: {e}")
        import traceback
        traceback.print_exc()  # Print the full traceback for debugging
        return None

# Main execution block
if __name__ == "__main__":
    # Set parameters
    results_directory = "results"  # Directory containing patient subdirectories
    min_read_coverage = 1         # Minimum total reads required
    min_vaf_threshold = 0.0       # Minimum VAF required (0%)
    missing_ref_value = 100       # Reference count for variants not detected in a sample
    
    # Get all patient variant files
    print(f"Finding patient variant files in {results_directory}")
    patient_files = get_patient_variant_files(results_dir=results_directory)
    
    if not patient_files:
        print("No patient variant files found. Exiting.")
        exit(1)
    
    # Process each patient independently
    clonefinder_results = {}
    for patient_id, variant_file in patient_files.items():
        clonefinder_df = process_patient_variants_for_clonefinder(
            patient_id,
            variant_file,
            min_coverage=min_read_coverage,
            min_vaf=min_vaf_threshold,
            use_simple_ids=True,                # Use simple S1, S2, etc. IDs
            require_all_samples=False,          # Include variants not found in all samples
            missing_ref_count=missing_ref_value # Reference count placeholder for missing variants
        )
        
        if clonefinder_df is not None:
            clonefinder_results[patient_id] = clonefinder_df
    
    # Display summary
    print("\nCloneFinder conversion complete:")
    for patient, data in clonefinder_results.items():
        sample_count = len([col for col in data.columns if ':ref' in col])
        print(f"  Patient {patient}: {len(data)} variants processed across {sample_count} samples")
        
    # Display the first few rows of the first patient's CloneFinder input
    if clonefinder_results:
        first_patient = list(clonefinder_results.keys())[0]
        print(f"\nFirst few rows of CloneFinder input for patient {first_patient}:")
        print(clonefinder_results[first_patient].head())

Finding patient variant files in results
Found variant file for patient p191: results/p191/p191_variants.tsv
Found variant file for patient P163: results/P163/P163_variants.tsv
Found variant file for patient P190: results/P190/P190_variants.tsv
Found variant file for patient p152: results/p152/p152_variants.tsv
Found variant file for patient M36: results/M36/M36_variants.tsv
Found variant file for patient G53: results/G53/G53_variants.tsv
Found variant file for patient G39: results/G39/G39_variants.tsv
Found variant file for patient G97: results/G97/G97_variants.tsv
Found variant file for patient G52: results/G52/G52_variants.tsv
Found variant file for patient G99: results/G99/G99_variants.tsv
Found variant file for patient G77: results/G77/G77_variants.tsv
Found variant file for patient G12: results/G12/G12_variants.tsv
Found variant file for patient M24: results/M24/M24_variants.tsv
Found variant file for patient G23: results/G23/G23_variants.tsv
Found variant file for patient G40: r

KeyboardInterrupt: 

### faster version

In [166]:
# VCF to CloneFinder Converter - Optimized Version
# This script reads variant data from TSV files in the results directory
# and saves CloneFinder input files in the same patient directories

import pandas as pd
import os
import glob
import numpy as np
import time

def get_patient_variant_files(results_dir="results", skip_existing=True):
    """
    Find all patient variant TSV files in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        skip_existing (bool): Skip patients with existing CloneFinder input files
        
    Returns:
        dict: Dictionary mapping patient IDs to their variant file paths
    """
    patient_files = {}
    skipped_count = 0
    
    # Look for patient directories in the results directory
    patient_dirs = glob.glob(os.path.join(results_dir, "*"))
    
    for patient_dir in patient_dirs:
        if os.path.isdir(patient_dir):
            # Extract patient ID from directory name
            patient_id = os.path.basename(patient_dir)
            
            # Check if CloneFinder file already exists
            clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_input.txt")
            if skip_existing and os.path.exists(clonefinder_file):
                skipped_count += 1
                continue
            
            # Look for variant TSV file with patient ID prefix
            variant_file_pattern = os.path.join(patient_dir, f"{patient_id}_variants.tsv")
            variant_files = glob.glob(variant_file_pattern)
            
            if variant_files:
                patient_files[patient_id] = variant_files[0]
                print(f"Found variant file for patient {patient_id}: {variant_files[0]}")
    
    if skipped_count > 0:
        print(f"Skipped {skipped_count} patients with existing CloneFinder input files")
    
    if not patient_files:
        print(f"No patient variant files found in {results_dir} that need processing")
    
    return patient_files

def process_patient_variants_for_clonefinder(patient_id, variant_file, 
                                           min_coverage=20, min_vaf=0.05,
                                           use_simple_ids=False, 
                                           require_all_samples=False,
                                           missing_ref_count=100):
    """
    Process a single patient's variants and convert to CloneFinder format
    
    Args:
        patient_id (str): Patient identifier
        variant_file (str): Path to patient's variant TSV file
        min_coverage (int): Minimum read coverage to include a variant
        min_vaf (float): Minimum VAF to include a variant
        use_simple_ids (bool): If True, generate simple SNV IDs (S1, S2, etc.)
        require_all_samples (bool): If True, only include variants found in all samples
        missing_ref_count (int): Reference count to use for variants not found in a sample
        
    Returns:
        pd.DataFrame: CloneFinder formatted data for the patient
    """
    start_time = time.time()
    output_dir = os.path.dirname(variant_file)
    output_file = os.path.join(output_dir, f"{patient_id}_clonefinder_input.txt")
    
    print(f"Processing patient {patient_id}...")
    
    try:
        # Read variant file - use optimized parameters
        variants_df = pd.read_csv(variant_file, sep='\t', 
                                 dtype={
                                     'CHROM': str, 
                                     'POS': str, 
                                     'REF': str, 
                                     'ALT': str, 
                                     'SAMPLE': str,
                                     'REF_COUNT': 'Int64',  # Changed to integer type
                                     'ALT_COUNT': 'Int64'   # Changed to integer type
                                 })
        print(f"  Loaded {len(variants_df)} variants")
        
        # Check for required columns
        required_cols = ['CHROM', 'POS', 'REF', 'ALT', 'REF_COUNT', 'ALT_COUNT', 'SAMPLE']
        missing_columns = [col for col in required_cols if col not in variants_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns in variant file: {', '.join(missing_columns)}")
        
        # Convert any float counts to integers if they weren't already loaded as integers
        if not pd.api.types.is_integer_dtype(variants_df['REF_COUNT']):
            variants_df['REF_COUNT'] = variants_df['REF_COUNT'].round().astype('Int64')
        
        if not pd.api.types.is_integer_dtype(variants_df['ALT_COUNT']):
            variants_df['ALT_COUNT'] = variants_df['ALT_COUNT'].round().astype('Int64')
        
        # Filter variants by coverage and VAF - vectorized operations
        if 'TOTAL_COUNT' in variants_df.columns:
            total_count = variants_df['TOTAL_COUNT']
            if not pd.api.types.is_integer_dtype(total_count):
                total_count = total_count.round().astype('Int64')
        else:
            total_count = (variants_df['REF_COUNT'] + variants_df['ALT_COUNT']).astype('Int64')
            
        if 'VAF' in variants_df.columns:
            vaf = variants_df['VAF']
        else:
            # Fast vectorized calculation with safe division - convert to float for division
            vaf = variants_df['ALT_COUNT'].astype(float) / np.maximum(total_count.astype(float), 1)  # Avoid division by zero
        
        # Apply filters in a single step
        filtered_variants = variants_df[
            (total_count >= min_coverage) & 
            (vaf >= min_vaf)
        ].copy()
        
        print(f"  Filtered from {len(variants_df)} to {len(filtered_variants)} variants")
        print(f"    Minimum coverage: {min_coverage}")
        print(f"    Minimum VAF: {min_vaf}")
        
        if len(filtered_variants) == 0:
            raise ValueError("No variants remain after filtering. Consider lowering thresholds.")
        
        # Get list of unique samples for this patient
        samples = sorted(filtered_variants['SAMPLE'].unique())
        print(f"  Found {len(samples)} samples for patient {patient_id}: {', '.join(samples)}")
        
        # Create mutation IDs
        if use_simple_ids:
            # Create unique variants array using pandas operations instead of loops
            unique_variants = filtered_variants.drop_duplicates(['CHROM', 'POS', 'REF', 'ALT'])
            
            # Create efficient ID mapping
            unique_variants = unique_variants.reset_index(drop=True)
            unique_variants['SNVID'] = [f"S{i+1}" for i in range(len(unique_variants))]
            
            # Create a temporary key for joining
            filtered_variants['temp_key'] = filtered_variants['CHROM'] + ":" + \
                                          filtered_variants['POS'] + ":" + \
                                          filtered_variants['REF'] + ":" + \
                                          filtered_variants['ALT']
            unique_variants['temp_key'] = unique_variants['CHROM'] + ":" + \
                                        unique_variants['POS'] + ":" + \
                                        unique_variants['REF'] + ":" + \
                                        unique_variants['ALT']
            
            # Join to transfer SNVIDs to filtered_variants
            id_map = unique_variants[['temp_key', 'SNVID']].set_index('temp_key')['SNVID']
            filtered_variants['SNVID'] = filtered_variants['temp_key'].map(id_map)
            filtered_variants.drop('temp_key', axis=1, inplace=True)
            
            snv_id_column = 'SNVID'
        else:
            print("  Creating mutation IDs based on genomic coordinates")
            # Vectorized string concatenation
            filtered_variants['SNVID'] = filtered_variants['CHROM'] + ":" + \
                                       filtered_variants['POS'] + ":" + \
                                       filtered_variants['REF'] + ">" + \
                                       filtered_variants['ALT']
            snv_id_column = 'SNVID'
        
        # Find variants across samples based on setting
        if len(samples) > 1:
            # Count number of samples per variant
            variant_counts = filtered_variants.groupby(snv_id_column)['SAMPLE'].nunique()
            
            if require_all_samples:
                # Only keep variants present in all samples
                common_variants = variant_counts[variant_counts == len(samples)].index.tolist()
                
                if len(common_variants) == 0:
                    print(f"  Warning: No variants found in all samples for patient {patient_id}. Consider setting require_all_samples=False")
                    if len(variant_counts) > 0:
                        # Find variants in the maximum number of samples
                        max_samples = variant_counts.max()
                        common_variants = variant_counts[variant_counts == max_samples].index.tolist()
                        print(f"  Including {len(common_variants)} variants found in {max_samples}/{len(samples)} samples")
                else:
                    print(f"  Found {len(common_variants)} variants present in all {len(samples)} samples")
            else:
                # Keep all variants
                common_variants = filtered_variants[snv_id_column].unique()
                print(f"  Including all {len(common_variants)} variants (may not be present in all samples)")
                
            filtered_variants = filtered_variants[filtered_variants[snv_id_column].isin(common_variants)]
        
        # Create a reference for the alleles (using pandas operations)
        variant_alleles = filtered_variants[[snv_id_column, 'REF', 'ALT']].drop_duplicates(snv_id_column)
        variant_alleles_dict = {
            'REF': dict(zip(variant_alleles[snv_id_column], variant_alleles['REF'])),
            'ALT': dict(zip(variant_alleles[snv_id_column], variant_alleles['ALT']))
        }
        
        # Get all variants and samples
        all_variant_ids = filtered_variants[snv_id_column].unique()
        
        # Optimization: Use a dictionary to store counts for faster lookup
        counts_dict = {}
        for _, row in filtered_variants.iterrows():
            key = (row[snv_id_column], row['SAMPLE'])
            counts_dict[key] = (int(row['REF_COUNT']), int(row['ALT_COUNT']))  # Explicitly convert to int
        
        # Create a complete matrix of variants and samples
        result_data = []
        for var_id in all_variant_ids:
            row_data = {snv_id_column: var_id, 
                      'Wild': variant_alleles_dict['REF'][var_id], 
                      'Mut': variant_alleles_dict['ALT'][var_id]}
            
            # Add reference and alternate counts for each sample
            for sample in samples:
                key = (var_id, sample)
                if key in counts_dict:
                    ref_count, alt_count = counts_dict[key]
                else:
                    ref_count, alt_count = missing_ref_count, 0
                
                row_data[f"{sample}:ref"] = int(ref_count)  # Ensure integer values
                row_data[f"{sample}:alt"] = int(alt_count)  # Ensure integer values
            
            result_data.append(row_data)
        
        # Create DataFrame from result data
        clonefinder_data = pd.DataFrame(result_data)
        
        # Ensure all count columns are integers
        for col in clonefinder_data.columns:
            if ':ref' in col or ':alt' in col:
                clonefinder_data[col] = clonefinder_data[col].astype(int)
        
        # Reorder columns to match expected format
        ref_cols = [f"{sample}:ref" for sample in samples]
        alt_cols = [f"{sample}:alt" for sample in samples]
        cols = [snv_id_column, 'Wild', 'Mut'] + ref_cols + alt_cols
        
        # Select and order columns
        existing_cols = [col for col in cols if col in clonefinder_data.columns]
        clonefinder_data = clonefinder_data[existing_cols]
        
        # Check and add any missing columns
        for col in cols:
            if col not in clonefinder_data.columns:
                if ':ref' in col:
                    clonefinder_data[col] = int(missing_ref_count)
                elif ':alt' in col:
                    clonefinder_data[col] = 0
        
        # Final reordering
        clonefinder_data = clonefinder_data[cols]
        
        # Save to file with integer format (no decimal places)
        clonefinder_data.to_csv(output_file, sep='\t', index=False, float_format='%d')
        end_time = time.time()
        print(f"  CloneFinder input saved to {output_file}")
        print(f"  Processing time: {end_time - start_time:.2f} seconds")
        
        return clonefinder_data
        
    except Exception as e:
        end_time = time.time()
        print(f"  Error processing patient {patient_id}: {e}")
        print(f"  Processing time: {end_time - start_time:.2f} seconds")
        import traceback
        traceback.print_exc()  # Print the full traceback for debugging
        return None


def process_patient_parallel(args):
    """Wrapper for parallel processing"""
    patient_id, variant_file, params = args
    return process_patient_variants_for_clonefinder(
        patient_id=patient_id,
        variant_file=variant_file,
        **params
    )

# Main execution block
if __name__ == "__main__":
    start_time = time.time()
    # Set parameters
    results_directory = "results"  # Directory containing patient subdirectories
    min_read_coverage = 1         # Minimum total reads required
    min_vaf_threshold = 0.0       # Minimum VAF required (0%)
    missing_ref_value = 100       # Reference count for variants not detected in a sample
    skip_existing_files = True    # Skip patients with existing CloneFinder input files
    use_parallel = False          # Disable parallel processing by default due to potential issues
    max_workers = 4               # Number of parallel workers
    
    # Process parameters
    processing_params = {
        'min_coverage': min_read_coverage,
        'min_vaf': min_vaf_threshold,
        'use_simple_ids': True,             # Use simple S1, S2, etc. IDs
        'require_all_samples': False,       # Include variants not found in all samples
        'missing_ref_count': missing_ref_value # Reference count placeholder for missing variants
    }
    
    # Get all patient variant files, skipping those with existing output files
    print(f"Finding patient variant files in {results_directory}")
    patient_files = get_patient_variant_files(
        results_dir=results_directory,
        skip_existing=skip_existing_files
    )
    
    if not patient_files:
        print("No patient variant files found that need processing. Exiting.")
        exit(0)
    
    # Process each patient
    clonefinder_results = {}
    
    # Use sequential processing to avoid multiprocessing issues
    print("Using sequential processing")
    for patient_id, variant_file in patient_files.items():
        clonefinder_df = process_patient_variants_for_clonefinder(
            patient_id,
            variant_file,
            **processing_params
        )
        
        if clonefinder_df is not None:
            clonefinder_results[patient_id] = clonefinder_df
    
    # Display summary
    end_time = time.time()
    print("\nCloneFinder conversion complete:")
    for patient, data in clonefinder_results.items():
        sample_count = len([col for col in data.columns if ':ref' in col])
        print(f"  Patient {patient}: {len(data)} variants processed across {sample_count} samples")
    
    print(f"\nTotal processing time: {end_time - start_time:.2f} seconds")
    
    # Display the first few rows of the first patient's CloneFinder input
    if clonefinder_results:
        first_patient = list(clonefinder_results.keys())[0]
        print(f"\nFirst few rows of CloneFinder input for patient {first_patient}:")
        print(clonefinder_results[first_patient].head())

Finding patient variant files in results
Found variant file for patient p065: results/p065/p065_variants.tsv
Found variant file for patient p129: results/p129/p129_variants.tsv
Skipped 26 patients with existing CloneFinder input files
Using sequential processing
Processing patient p065...
  Loaded 37838 variants
  Filtered from 37838 to 37838 variants
    Minimum coverage: 1
    Minimum VAF: 0.0
  Found 6 samples for patient p065: 358799, 358802, 359303, 359306, 359319, 359322
  Including all 37341 variants (may not be present in all samples)
  CloneFinder input saved to results/p065/p065_clonefinder_input.txt
  Processing time: 0.92 seconds
Processing patient p129...
  Loaded 5492 variants
  Filtered from 5492 to 5492 variants
    Minimum coverage: 1
    Minimum VAF: 0.0
  Found 10 samples for patient p129: 375688, 375918, 375970, 375973, 376050, 376085, 376165, 376167, 376176, 376304
  Including all 4061 variants (may not be present in all samples)
  CloneFinder input saved to result

## LICHeE

### Convert the VAF data into the format expected by LICHeE

In [233]:
# VCF to LICHeE Converter
# This script reads variant data from TSV files in the results directory
# and saves LICHeE input files in the same patient directories

import pandas as pd
import os
import glob
import numpy as np

def get_patient_variant_files(results_dir="results", specific_patient=None):
    """
    Find all patient variant TSV files in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        specific_patient (str, optional): Specific patient ID to process
        
    Returns:
        dict: Dictionary mapping patient IDs to their variant file paths
    """
    patient_files = {}
    
    # Look for patient directories in the results directory
    patient_dirs = glob.glob(os.path.join(results_dir, "*"))
    
    for patient_dir in patient_dirs:
        if os.path.isdir(patient_dir):
            # Extract patient ID from directory name
            patient_id = os.path.basename(patient_dir)
            
            # Skip if specific patient is requested and this isn't it
            if specific_patient and patient_id != specific_patient:
                continue
                
            # Look for variant TSV file with patient ID prefix
            variant_file_pattern = os.path.join(patient_dir, f"{patient_id}_variants.tsv")
            variant_files = glob.glob(variant_file_pattern)
            
            if variant_files:
                patient_files[patient_id] = variant_files[0]
                print(f"Found variant file for patient {patient_id}: {variant_files[0]}")
    
    if not patient_files:
        if specific_patient:
            print(f"No variant file found for patient {specific_patient} in {results_dir}")
        else:
            print(f"No patient variant files found in {results_dir}")
    
    return patient_files

def process_patient_variants_for_lichee(patient_id, variant_file, 
                                      min_coverage=20, min_vaf=0.05,
                                      normal_sample=None, normal_vaf=0.0,
                                      gene_column=None):
    """
    Process a single patient's variants and convert to LICHeE format
    
    Args:
        patient_id (str): Patient identifier
        variant_file (str): Path to patient's variant TSV file
        min_coverage (int): Minimum read coverage to include a variant
        min_vaf (float): Minimum VAF to include a variant
        normal_sample (str, optional): Name of normal sample if included in data
        normal_vaf (float): VAF value to use for normal sample if not in data
        gene_column (str, optional): Column name containing gene names
        
    Returns:
        pd.DataFrame: LICHeE formatted data for the patient
    """
    output_dir = os.path.dirname(variant_file)
    output_file = os.path.join(output_dir, f"{patient_id}_lichee_input.txt")
    
    print(f"Processing patient {patient_id}...")
    print(f"  Using minimum coverage: {min_coverage}")
    print(f"  Using minimum VAF: {min_vaf}")
    
    try:
        # Read variant file
        variants_df = pd.read_csv(variant_file, sep='\t')
        print(f"  Loaded {len(variants_df)} variants")
        
        # Check for required columns
        required_cols = ['CHROM', 'POS', 'REF', 'ALT', 'REF_COUNT', 'ALT_COUNT', 'SAMPLE']
        missing_columns = [col for col in required_cols if col not in variants_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns in variant file: {', '.join(missing_columns)}")
        
        # Ensure string columns are strings and numeric columns are numeric
        variants_df['CHROM'] = variants_df['CHROM'].astype(str)
        variants_df['POS'] = pd.to_numeric(variants_df['POS'])
        variants_df['REF'] = variants_df['REF'].astype(str)
        variants_df['ALT'] = variants_df['ALT'].astype(str)
        variants_df['SAMPLE'] = variants_df['SAMPLE'].astype(str)
        variants_df['REF_COUNT'] = pd.to_numeric(variants_df['REF_COUNT'])
        variants_df['ALT_COUNT'] = pd.to_numeric(variants_df['ALT_COUNT'])
        
        # Calculate VAF if not provided
        if 'VAF' not in variants_df.columns:
            print("  Calculating VAF from REF_COUNT and ALT_COUNT")
            total_count = variants_df['REF_COUNT'] + variants_df['ALT_COUNT']
            variants_df['VAF'] = variants_df['ALT_COUNT'] / total_count.replace(0, 1)  # Avoid division by zero
        else:
            variants_df['VAF'] = pd.to_numeric(variants_df['VAF'])
        
        # Filter variants by coverage and VAF and not mitochondrial DNA
        total_count = variants_df['REF_COUNT'] + variants_df['ALT_COUNT']
        
        filtered_variants = variants_df[
            (total_count >= min_coverage) & 
            (variants_df['VAF'] >= min_vaf) &
            (variants_df['CHROM'] != 'chrM')
        ].copy()
        
        print(f"  Filtered from {len(variants_df)} to {len(filtered_variants)} variants")
        
        if len(filtered_variants) == 0:
            raise ValueError("No variants remain after filtering. Consider lowering thresholds.")
        
        # Get list of unique samples for this patient
        samples = sorted(filtered_variants['SAMPLE'].unique().astype(str))
        
        # Remove normal sample from tumor sample list if specified
        if normal_sample and normal_sample in samples:
            samples.remove(normal_sample)
            print(f"  Found normal sample: {normal_sample}")
        
        print(f"  Found {len(samples)} tumor samples for patient {patient_id}: {', '.join(samples)}")
        
        # Create a description column
        if gene_column and gene_column in filtered_variants.columns:
            # Use gene name if available
            filtered_variants['description'] = filtered_variants.apply(
                lambda row: f"{row['REF']}/{row['ALT']} {str(row[gene_column])}", axis=1
            )
        else:
            # Just use the allele change
            filtered_variants['description'] = filtered_variants.apply(
                lambda row: f"{row['REF']}/{row['ALT']}", axis=1
            )
        
        # Convert chromosome format if needed (ensure "chr" prefix is removed)
        filtered_variants['clean_chrom'] = filtered_variants['CHROM'].str.replace('chr', '')
        
        # Create a unique variant ID from chromosome and position
        filtered_variants['var_id'] = filtered_variants.apply(
            lambda row: f"{row['clean_chrom']}_{row['POS']}_{row['REF']}_{row['ALT']}", axis=1
        )
        
        # Extract unique variants (same chr, pos, ref, alt may appear in multiple samples)
        unique_variants = filtered_variants.drop_duplicates('var_id')
        
        # Create a pivot table with VAFs for each sample
        pivot_vaf = filtered_variants.pivot_table(
            index='var_id', 
            columns='SAMPLE', 
            values='VAF', 
            aggfunc='mean'  # Use mean in case of duplicates
        ).fillna(0)  # Fill missing values with 0 (variant not found in that sample)
        
        # Get chromosome, position, and description for each variant
        var_info = unique_variants.set_index('var_id')[['clean_chrom', 'POS', 'description']]
        
        # Merge VAFs with variant info
        lichee_data = var_info.join(pivot_vaf).reset_index()
        
        # Create output DataFrame in LICHeE format
        output_columns = ['clean_chrom', 'POS', 'description']
        
        # Add normal sample column if needed
        if normal_sample and normal_sample in pivot_vaf.columns:
            # Normal sample exists in the data
            output_columns.append(normal_sample)
        else:
            # Add a placeholder normal sample with VAF 0.0
            lichee_data['Normal'] = normal_vaf
            output_columns.append('Normal')
        
        # Add tumor sample columns
        output_columns.extend(samples)
        
        # Select and rename columns
        lichee_data = lichee_data[output_columns]
        lichee_data.columns = ['#chr', 'position', 'description'] + ([normal_sample] if normal_sample in pivot_vaf.columns else ['Normal']) + samples
        
        # Convert numeric values to appropriate format
        lichee_data['position'] = lichee_data['position'].astype(int)
        for sample in samples + (['Normal'] if 'Normal' in lichee_data.columns else [normal_sample]):
            if sample is not None:  # Skip None values
                lichee_data[sample] = lichee_data[sample].round(2)
        
        # Save to file
        lichee_data.to_csv(output_file, sep='\t', index=False, float_format='%.2f')
        print(f"  LICHeE input saved to {output_file}")
        
        return lichee_data
        
    except Exception as e:
        print(f"  Error processing patient {patient_id}: {e}")
        import traceback
        traceback.print_exc()  # Print the full traceback for debugging
        return None


results_directory = "results"  # Directory containing patient subdirectories
min_read_coverage = 1         # Default minimum total reads required
min_vaf_threshold = 0.05       # Minimum VAF required (5%)
normal_sample = None           # Specify normal sample name if in your data
normal_vaf = 0.0               # VAF to use for normal sample (usually 0.0)
gene_column = None             # Column with gene names (if available)

# Create a dictionary for patient-specific min_coverage values
patient_coverage_map = {
    "G23": 20,
    "G43": 20,
    "G53": 20,
    "G77": 20,
    "G87": 20,
    "G97": 20,
    "G99": 400,
    "M95": 20,
    "p129": 20,
    "P163": 20,
    "P190": 20
}

# Specify a single patient ID to process (set to None to process all patients)
specific_patient_id = "G99" 

# Get all patient variant files (or specific patient if requested)
print(f"Finding patient variant files in {results_directory}")
patient_files = get_patient_variant_files(
    results_dir=results_directory,
    specific_patient=specific_patient_id
)

if not patient_files:
    print("No patient variant files found. Exiting.")
else:
    # Process each patient independently
    lichee_results = {}
    for patient_id, variant_file in patient_files.items():
        # Use patient-specific coverage if available, otherwise use default
        patient_min_coverage = patient_coverage_map.get(patient_id, min_read_coverage)
        
        lichee_df = process_patient_variants_for_lichee(
            patient_id,
            variant_file,
            min_coverage=patient_min_coverage,
            min_vaf=min_vaf_threshold,
            normal_sample=normal_sample,
            normal_vaf=normal_vaf,
            gene_column=gene_column
        )
        
        if lichee_df is not None:
            lichee_results[patient_id] = lichee_df

Finding patient variant files in results
Found variant file for patient G99: results/G99/G99_variants.tsv
Processing patient G99...
  Using minimum coverage: 400
  Using minimum VAF: 0.05
  Loaded 45509 variants
  Filtered from 45509 to 755 variants
  Found 4 tumor samples for patient G99: E, F, I, K
  LICHeE input saved to results/G99/G99_lichee_input.txt


### Run the tool

./lichee -build -i ../../../lichee_input.txt -maxVAFAbsent 0.005 -minVAFPresent 0.005 -n 0 -showTree 1 -s 1 -v

run the following from the lichee release folder

need to leave off the -showTree 1 -s 1 for the file to generate

i also had to use a high min read coverage threshold (150)

./lichee -build -i ../../../lichee_input.txt -maxVAFAbsent 0.005 -minVAFPresent 0.005 -n 0 -minClusterSize 20 -minPrivateClusterSize 20

i also had to rebuild the release jar according to the suggestions in this post https://github.com/viq854/lichee/issues/14



## Run each tool

In [ ]:
import os
import subprocess
import glob
import shutil
from pathlib import Path
import pandas as pd

def run_pyclone(patient_id, input_file, output_dir, chains=40, density="beta-binomial", restarts=10):
    """
    Run PyClone-VI for a patient
    
    Args:
        patient_id (str): Patient identifier
        input_file (str): Path to PyClone-VI input file
        output_dir (str): Directory to save results
        chains (int): Number of MCMC chains
        density (str): Density to use (beta-binomial or binomial)
        restarts (int): Number of restarts
    
    Returns:
        bool: True if successful, False otherwise
    """
    print(f"Running PyClone-VI for patient {patient_id}...")
    
    # Create output file path
    output_h5 = os.path.join(output_dir, f"{patient_id}_pyclone_results.h5")
    output_tsv = os.path.join(output_dir, f"{patient_id}_pyclone_results.tsv")
    
    # Construct command for fit
    fit_cmd = [
        "pyclone-vi", "fit",
        "-i", input_file,
        "-o", output_h5,
        "-c", str(chains),
        "-d", density,
        "-r", str(restarts)
    ]
    
    print(f"  Fit command: {' '.join(fit_cmd)}")
    
    try:
        # Run fit command
        result = subprocess.run(fit_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"  PyClone-VI fit completed successfully: {output_h5}")
        
        # Construct command for write-results-file
        write_cmd = [
            "pyclone-vi", "write-results-file",
            "-i", output_h5,
            "-o", output_tsv
        ]
        
        print(f"  Write results command: {' '.join(write_cmd)}")
        
        # Run write-results-file command
        write_result = subprocess.run(write_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"  PyClone-VI results written to TSV: {output_tsv}")
        
        return True
    except subprocess.CalledProcessError as e:
        print(f"  Error running PyClone-VI: {e}")
        print(f"  stderr: {e.stderr}")
        return False

def run_clonefinder(patient_id, input_file, output_dir, clonefinder_path):
    """
    Run CloneFinder for a patient
    
    Args:
        patient_id (str): Patient identifier
        input_file (str): Path to CloneFinder input file
        output_dir (str): Directory to save results
        clonefinder_path (str): Path to CloneFinder directory
    
    Returns:
        bool: True if successful, False otherwise
    """
    print(f"Running CloneFinder for patient {patient_id}...")
    
    # Get the basename of the input file
    input_basename = os.path.basename(input_file)
    
    # Copy input file to CloneFinder directory (required by CloneFinder)
    temp_input = os.path.join(clonefinder_path, input_basename)
    shutil.copy2(input_file, temp_input)
    
    # Construct command
    cmd = [
        "python3", "clonefinder.py", 
        "snv", input_basename
    ]
    
    print(f"  Command: {' '.join(cmd)}")
    
    try:
        # Run command from CloneFinder directory
        result = subprocess.run(cmd, cwd=clonefinder_path, check=True, 
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Move output files to patient directory
        output_files = glob.glob(os.path.join(clonefinder_path, f"{input_basename}*"))
        for file in output_files:
            if os.path.isfile(file):
                dest_file = os.path.join(output_dir, os.path.basename(file))
                shutil.move(file, dest_file)
                print(f"  Moved output file to: {dest_file}")
        
        # Move any other CloneFinder output files that might not match the input filename pattern
        # (e.g., more general result files that CloneFinder might generate)
        additional_outputs = glob.glob(os.path.join(clonefinder_path, f"*{patient_id}*"))
        for file in additional_outputs:
            if os.path.isfile(file):
                dest_file = os.path.join(output_dir, os.path.basename(file))
                shutil.move(file, dest_file)
                print(f"  Moved additional output file to: {dest_file}")
        
        print(f"  CloneFinder completed successfully: {output_dir}")
        
        # Clean up temporary input file
        if os.path.exists(temp_input):
            os.remove(temp_input)
            
        return True
    except subprocess.CalledProcessError as e:
        print(f"  Error running CloneFinder: {e}")
        print(f"  stderr: {e.stderr}")
        
        # Clean up temporary input file
        if os.path.exists(temp_input):
            os.remove(temp_input)
            
        return False


def run_lichee_tool(patient_id, input_file, output_dir, lichee_path, 
              max_vaf_absent=0.005, min_vaf_present=0.005, 
              normal_sample_idx=0, min_cluster_size=20, min_private_cluster_size=20):
    """
    Run LICHeE for a patient
    
    Args:
        patient_id (str): Patient identifier
        input_file (str): Path to LICHeE input file
        output_dir (str): Directory to save results
        lichee_path (str): Path to LICHeE executable
        max_vaf_absent (float): Maximum VAF to consider a variant absent
        min_vaf_present (float): Minimum VAF to consider a variant present
        normal_sample_idx (int): Index of normal sample (0-based)
        min_cluster_size (int): Minimum cluster size
        min_private_cluster_size (int): Minimum private cluster size
    
    Returns:
        bool: True if successful, False otherwise
    """
    print(f"Running LICHeE for patient {patient_id}...")
    
    # Create output directory path
    patient_output_dir = os.path.join(output_dir, f"{patient_id}_lichee_results")
    os.makedirs(patient_output_dir, exist_ok=True)
    
    # Construct command
    cmd = [
        "./lichee",
        "-build",
        "-i", input_file,
        "-maxVAFAbsent", str(max_vaf_absent),
        "-minVAFPresent", str(min_vaf_present),
        "-n", str(normal_sample_idx),
        "-minClusterSize", str(min_cluster_size),
        "-minPrivateClusterSize", str(min_private_cluster_size),
        "-o", os.path.join(patient_output_dir, f"{patient_id}_lichee_output")
    ]
    
    print(f"  Command: {' '.join(cmd)}")
    
    try:
        # Run command
        # result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # /opt/homebrew/opt/openjdk/bin/java
        env = os.environ.copy()
        result = subprocess.run(cmd, cwd=lichee_path, check=True, shell=True, env=env,
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"  LICHeE completed successfully: {patient_output_dir}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"  Error running LICHeE: {e}")
        print(f"  stderr: {e.stderr}")
        return False


def run_tools_for_patient(patient_id, results_dir="results", 
                         clonefinder_path="CloneFinder",
                         lichee_path="./lichee",
                         run_pyclone_vi=True, run_clone_finder=True, run_lichee=True):
    """
    Run selected tools for a single patient
    
    Args:
        patient_id (str): Patient identifier
        results_dir (str): Base directory for results
        clonefinder_path (str): Path to CloneFinder directory
        run_pyclone_vi (bool): Whether to run PyClone-VI
        run_clone_finder (bool): Whether to run CloneFinder
    
    Returns:
        dict: Dictionary with results for each tool
    """
    # Get patient directory
    patient_dir = os.path.join(results_dir, patient_id)
    
    if not os.path.isdir(patient_dir):
        print(f"Error: Patient directory '{patient_dir}' not found.")
        return {}
    
    # Find input files
    pyclone_input = os.path.join(patient_dir, f"{patient_id}_pyclone_input.tsv")
    clonefinder_input = os.path.join(patient_dir, f"{patient_id}_clonefinder_input.txt")
    lichee_input = os.path.join(patient_dir, f"{patient_id}_lichee_input.txt")

    # Check if input files exist and run tools
    results = {}
    
    if run_pyclone_vi:
        if os.path.isfile(pyclone_input):
            results['pyclone'] = run_pyclone(patient_id, pyclone_input, patient_dir)
        else:
            print(f"Warning: PyClone-VI input file '{pyclone_input}' not found.")
            results['pyclone'] = False
    
    if run_clone_finder:
        if os.path.isfile(clonefinder_input):
            results['clonefinder'] = run_clonefinder(patient_id, clonefinder_input, patient_dir, clonefinder_path)
        else:
            print(f"Warning: CloneFinder input file '{clonefinder_input}' not found.")
            results['clonefinder'] = False
    
    if run_lichee:
        if os.path.isfile(lichee_input):
            results['lichee'] = run_lichee_tool(patient_id, lichee_input, patient_dir, lichee_path)
        else:
            print(f"Warning: LICHeE input file '{lichee_input}' not found.")
            results['lichee'] = False
    
    return results


def find_all_patients(results_dir="results"):
    """
    Find all patient directories in the results directory
    
    Args:
        results_dir (str): Path to the results directory
        
    Returns:
        list: List of patient IDs
    """
    patient_dirs = [os.path.basename(d) for d in glob.glob(os.path.join(results_dir, "*")) 
                   if os.path.isdir(d)]
    return patient_dirs



In [432]:
# Set paths to tools
clonefinder_path = "CloneFinder"  # Change this to your CloneFinder directory
lichee_path="./lichee/LICHeE/release"

for patient_id in patient_ids:
    # Run PyClone-VI and CloneFinder for a specific patient
    patient_id = patient_id  # Change this to your patient ID
    results = run_tools_for_patient(
        patient_id, 
        results_dir="results",
        clonefinder_path=clonefinder_path,
        lichee_path=lichee_path,
        run_pyclone_vi=True,  # Set to False to skip PyClone-VI
        run_clone_finder=False,  # Set to False to skip CloneFinder
        run_lichee=False
    )

    # Display results
    for tool, success in results.items():
        status = "Success" if success else "Failed"
        print(f"{tool}: {status}")

Running PyClone-VI for patient G12...
  Fit command: pyclone-vi fit -i results/G12/G12_pyclone_input.tsv -o results/G12/G12_pyclone_results.h5 -c 40 -d beta-binomial -r 10
  PyClone-VI fit completed successfully: results/G12/G12_pyclone_results.h5
  Write results command: pyclone-vi write-results-file -i results/G12/G12_pyclone_results.h5 -o results/G12/G12_pyclone_results.tsv
  PyClone-VI results written to TSV: results/G12/G12_pyclone_results.tsv
pyclone: Success
Running PyClone-VI for patient G23...
  Fit command: pyclone-vi fit -i results/G23/G23_pyclone_input.tsv -o results/G23/G23_pyclone_results.h5 -c 40 -d beta-binomial -r 10
  PyClone-VI fit completed successfully: results/G23/G23_pyclone_results.h5
  Write results command: pyclone-vi write-results-file -i results/G23/G23_pyclone_results.h5 -o results/G23/G23_pyclone_results.tsv
  PyClone-VI results written to TSV: results/G23/G23_pyclone_results.tsv
pyclone: Success
Running PyClone-VI for patient G39...
  Fit command: pyclone

In [149]:
# !cd CloneFinder/ && python3 clonefinder.py snv ../results/G45/G45_clonefinder_input.txt

## Parse results and create visualizations for each tool

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# def parse_pyclone_results(results_file):
#     """
#     Parse PyClone-VI results into a format suitable for heatmap visualization,
#     using cellular prevalence values to represent clone frequencies.
    
#     Args:
#         results_file (str): Path to PyClone-VI results TSV file
    
#     Returns:
#         pandas.DataFrame: DataFrame with samples as rows and clones as columns
#     """
#     print(f"Parsing PyClone-VI results from: {results_file}")
    
#     # Load PyClone results
#     df = pd.read_csv(results_file, sep='\t')
    
#     # Get unique samples and clusters
#     samples = df['sample_id'].unique()
#     clusters = sorted(df['cluster_id'].unique())
    
#     print(f"Found {len(samples)} samples and {len(clusters)} clusters")
    
#     # Create a DataFrame to store mean cellular prevalence per cluster per sample
#     prev_matrix = pd.DataFrame(index=samples, columns=[f'Clone {c}' for c in clusters])
    
#     # For each sample and cluster, calculate the mean cellular prevalence
#     for sample in samples:
#         sample_df = df[df['sample_id'] == sample]
        
#         for cluster in clusters:
#             # Get cellular prevalence values for this sample and cluster
#             cluster_data = sample_df[sample_df['cluster_id'] == cluster]
            
#             if not cluster_data.empty:
#                 # Calculate mean cellular prevalence
#                 mean_prev = cluster_data['cellular_prevalence'].mean()
#                 prev_matrix.loc[sample, f'Clone {cluster}'] = mean_prev
#                 # print(f"Sample {sample}, Clone {cluster}: Mean prevalence = {mean_prev:.4f}")
#             else:
#                 prev_matrix.loc[sample, f'Clone {cluster}'] = 0.0
    
#     # Fill any NaN values with 0
#     prev_matrix = prev_matrix.fillna(0.0)
    
#     return prev_matrix

def parse_pyclone_results(results_file):
    """
    Parse PyClone-VI results into a format suitable for heatmap visualization,
    using cellular prevalence values to represent clone frequencies.
    
    Args:
        results_file (str): Path to PyClone-VI results TSV file
    
    Returns:
        pandas.DataFrame: DataFrame with samples as rows and clones as columns
    """
    print(f"Parsing PyClone-VI results from: {results_file}")
    
    # Load PyClone results
    df = pd.read_csv(results_file, sep='\t')
    
    # Get unique samples and clusters
    samples = df['sample_id'].unique()
    clusters = sorted(df['cluster_id'].unique())
    print(f"Found {len(samples)} samples and {len(clusters)} clusters")
    
    # Create a DataFrame to store clone frequencies
    freq_matrix = pd.DataFrame(index=samples, columns=[f'Clone {c}' for c in clusters])
    
    # Calculate frequencies for each sample
    for sample in samples:
        sample_df = df[df['sample_id'] == sample]
        
        # Filter clusters with cellular prevalence > 0 for this sample
        valid_clusters = {}
        
        for cluster in clusters:
            cluster_data = sample_df[sample_df['cluster_id'] == cluster]
            if not cluster_data.empty:
                mean_prevalence = cluster_data['cellular_prevalence'].mean()
                if mean_prevalence > 0:
                    valid_clusters[cluster] = mean_prevalence
        
        # Calculate total prevalence for normalization
        total_prevalence = sum(valid_clusters.values())
        
        # Calculate normalized frequencies
        if total_prevalence > 0:
            for cluster in clusters:
                if cluster in valid_clusters:
                    # Normalize by total prevalence to get frequency
                    freq = valid_clusters[cluster] / total_prevalence
                    freq_matrix.loc[sample, f'Clone {cluster}'] = freq
                else:
                    freq_matrix.loc[sample, f'Clone {cluster}'] = 0.0
        else:
            # Set all frequencies to 0 if no clusters have prevalence > 0
            for cluster in clusters:
                freq_matrix.loc[sample, f'Clone {cluster}'] = 0.0
    
    # Fill any NaN values with 0
    freq_matrix = freq_matrix.fillna(0.0)
    
    return freq_matrix

def parse_clonefinder_results(results_file):
    """
    Parse CloneFinder results into a format suitable for heatmap visualization
    
    Args:
        results_file (str): Path to CloneFinder results file
    
    Returns:
        pandas.DataFrame: DataFrame with samples as rows and clones as columns
    """
    print(f"Parsing CloneFinder results from: {results_file}")
    
    # Load CloneFinder results
    df = pd.read_csv(results_file, sep='\t')
    
    # The first column contains sample IDs, and other columns are clone frequencies
    # Rename the first column to 'Sample'
    df = df.rename(columns={df.columns[0]: 'Sample'})
    
    # Set Sample column as index
    df = df.set_index('Sample')
    
    # Sort columns by clone number
    # Extract numeric part from clone names and sort
    def get_clone_num(col_name):
        # Extract numeric part from strings like "Clone1", "Clone2", etc.
        if col_name.startswith('Clone'):
            try:
                return int(col_name[5:])
            except ValueError:
                return float('inf')  # Non-numeric parts go to the end
        else:
            try:
                # Handle cases like "1", "2", etc.
                return int(col_name)
            except ValueError:
                return float('inf')  # Non-numeric parts go to the end
    
    # Sort columns based on clone number
    sorted_cols = sorted(df.columns, key=get_clone_num)
    df = df[sorted_cols]
    
    # Convert all values to float (important for heatmap visualization)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

def parse_lichee_results(results_file):
    """
    Parse LICHeE results into a format suitable for heatmap visualization,
    correctly handling the hierarchical structure of clones
    
    Args:
        results_file (str): Path to LICHeE results file
    
    Returns:
        pandas.DataFrame: DataFrame with samples as rows and clones (nodes) as columns
    """
    print(f"Parsing LICHeE results from: {results_file}")
    
    # Initialize data structures
    lichee_data = {
        'nodes': {},          # Store node info: binary profile, mutations, etc.
        'tree': [],           # Store parent->child relationships
        'node_mapping': {},   # Map binary profiles to node IDs
        'sample_freqs': {}    # Store direct frequency of each node in each sample
    }
    
    try:
        with open(results_file, 'r') as f:
            content = f.read()
        
        # Parse Nodes section
        nodes_section = re.search(r'Nodes:(.*?)(?=\*\*\*\*Tree)', content, re.DOTALL)
        if nodes_section:
            nodes_text = nodes_section.group(1).strip()
            for line in nodes_text.split('\n'):
                if line.strip():
                    parts = line.split('\t')
                    if len(parts) >= 4:  # Node ID, binary profile, frequencies, mutations...
                        node_id = parts[0]
                        binary_profile = parts[1]
                        
                        # Extract frequencies from the format like [ 0.45 0.45 0.41 0.32]
                        frequencies_str = parts[2].strip('[]')
                        frequencies = [float(f) for f in frequencies_str.split() if f]
                        
                        # Extract mutations
                        mutations = parts[3:]
                        
                        lichee_data['nodes'][node_id] = {
                            'binary_profile': binary_profile,
                            'frequencies': frequencies,
                            'mutations': mutations
                        }
                        
                        # Map binary profile to node ID to help with tree parsing
                        if binary_profile not in lichee_data['node_mapping']:
                            lichee_data['node_mapping'][binary_profile] = []
                        lichee_data['node_mapping'][binary_profile].append(node_id)
            
            print(f"  Found {len(lichee_data['nodes'])} nodes")
            
        # Parse Tree section to get parent-child relationships
        tree_section = re.search(r'\*\*\*\*Tree.*?\*\*\*\*(.*?)(?=Sample decomposition:|$)', content, re.DOTALL)
        if tree_section:
            tree_text = tree_section.group(1).strip()
            for line in tree_text.split('\n'):
                if '->' in line:
                    parent, child = line.split('->')
                    lichee_data['tree'].append((parent.strip(), child.strip()))
            
            print(f"  Found {len(lichee_data['tree'])} parent-child relationships")
        
        # Parse Sample decomposition section
        sample_section = re.search(r'Sample decomposition:(.*?)(?=SNV info:|$)', content, re.DOTALL)
        if sample_section:
            sample_text = sample_section.group(1)
            
            # Initialize sample frequencies dictionary
            for node_id in lichee_data['nodes'].keys():
                lichee_data['sample_freqs'][f"Node {node_id}"] = {}
            
            # Parse each sample
            current_sample = None
            current_path = []  # Keep track of the hierarchical path
            
            lines = sample_text.split('\n')
            i = 0
            while i < len(lines):
                line = lines[i].strip()
                
                # Check for sample declaration
                if 'Sample lineage decomposition:' in line:
                    current_sample = line.split(':')[1].strip()
                    current_path = []
                    i += 1
                    continue
                
                # Skip empty lines or GL line
                if not line or line == 'GL':
                    i += 1
                    continue
                
                # Process lineage lines
                if line.startswith('.'):
                    # Determine the indentation level by counting dots
                    dot_count = len(line) - len(line.lstrip('.'))
                    
                    # Truncate path to the appropriate level
                    current_path = current_path[:dot_count]
                    
                    # Extract binary profile and frequency
                    match = re.search(r'\.+([01]+): ([0-9.]+)', line)
                    if match:
                        binary_profile = match.group(1)
                        frequency = float(match.group(2))
                        
                        # Find the corresponding node ID
                        if binary_profile in lichee_data['node_mapping']:
                            # If multiple nodes have the same binary profile, we need to determine which one
                            # is at this position in the hierarchy
                            matched_nodes = lichee_data['node_mapping'][binary_profile]
                            
                            # For now, just use the first matching node (simplification)
                            # In a more complex implementation, we would trace the tree structure
                            node_id = matched_nodes[0]
                            
                            # More sophisticated approach: try to find the correct node by checking the tree
                            if len(matched_nodes) > 1 and current_path:
                                for potential_node in matched_nodes:
                                    # Check if this node is a child of the last node in current_path
                                    if any(parent == current_path[-1].split()[1] and child == potential_node 
                                          for parent, child in lichee_data['tree']):
                                        node_id = potential_node
                                        break
                            
                            # Store the frequency for this node in this sample
                            node_key = f"Node {node_id}"
                            if current_sample not in lichee_data['sample_freqs'][node_key]:
                                lichee_data['sample_freqs'][node_key][current_sample] = frequency
                            
                            # Add this node to the current path
                            current_path.append(node_key)
                
                i += 1
            
            # Extract samples
            samples = [s for s in set([s for freqs in lichee_data['sample_freqs'].values() for s in freqs.keys()]) 
                      if s != 'Normal']
            print(f"  Found {len(samples)} samples: {', '.join(samples)}")
        
        # Create DataFrame from sample frequencies
        result_df = pd.DataFrame(index=samples)
        
        # Add columns for each node
        for node_key, freqs in lichee_data['sample_freqs'].items():
            result_df[node_key] = pd.Series({s: freqs.get(s, 0.0) for s in samples})
        
        # Fill NaN values with 0 (indicating absence of that clone in that sample)
        result_df = result_df.fillna(0)
        
        # Print the DataFrame for debugging
        print(f"  Final DataFrame shape: {result_df.shape}")
        
        return result_df
        
    except Exception as e:
        print(f"Error parsing LICHeE file: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# def plot_sample_bar_graphs(pyclone_data, clonefinder_data, lichee_data, patient_id, output_dir):
    """
    Create bar graphs for each sample showing clonal percentages from all three tools
    and save them all to a single file
    """
    # Get all sample IDs from all datasets
    all_samples = set()
    if pyclone_data is not None:
        all_samples.update(pyclone_data.index)
    if clonefinder_data is not None:
        all_samples.update(clonefinder_data.index)
    if lichee_data is not None:
        all_samples.update(lichee_data.index)
    
    # Convert all samples to strings to ensure consistent typing
    all_samples = {str(sample) for sample in all_samples}
    
    # Count the number of tools with data
    available_tools = sum([pyclone_data is not None, clonefinder_data is not None, lichee_data is not None])
    
    # Create a single figure with subplots for all samples and tools
    num_samples = len(all_samples)
    fig, axes = plt.subplots(num_samples, available_tools, 
                            figsize=(6 * available_tools, 4 * num_samples),
                            squeeze=False)
    
    # Sort samples for consistent ordering - now all are strings
    sorted_samples = sorted(all_samples)
    
    for sample_idx, sample in enumerate(sorted_samples):
        col_idx = 0
        
        # PyClone plot
        if pyclone_data is not None:
            # Check for both string and original type versions of the sample ID
            if sample in pyclone_data.index:
                sample_pyclone = pyclone_data.loc[sample]
            elif sample.isdigit() and int(sample) in pyclone_data.index:
                sample_pyclone = pyclone_data.loc[int(sample)]
            else:
                axes[sample_idx, col_idx].text(0.5, 0.5, "No PyClone-VI Data", 
                                            ha='center', va='center', fontsize=12)
                axes[sample_idx, col_idx].set_title(f"PyClone-VI: Sample {sample}", fontsize=12)
                col_idx += 1
                continue
                
            axes[sample_idx, col_idx].bar(sample_pyclone.index, sample_pyclone.values)
            axes[sample_idx, col_idx].set_title(f"PyClone-VI: Sample {sample}", fontsize=12)
            axes[sample_idx, col_idx].set_ylim(0, 1.0)
            plt.setp(axes[sample_idx, col_idx].get_xticklabels(), rotation=45, ha="right", 
                    rotation_mode="anchor", fontsize=10)
            
            # Only add y-label to the first column
            if col_idx == 0:
                axes[sample_idx, col_idx].set_ylabel("Frequency", fontsize=12)
                
            col_idx += 1
            
        # CloneFinder plot - similar pattern for the other tools
        if clonefinder_data is not None:
            # Same fix for CloneFinder
            if sample in clonefinder_data.index:
                sample_clonefinder = clonefinder_data.loc[sample]
            elif sample.isdigit() and int(sample) in clonefinder_data.index:
                sample_clonefinder = clonefinder_data.loc[int(sample)]
            else:
                axes[sample_idx, col_idx].text(0.5, 0.5, "No CloneFinder Data", 
                                            ha='center', va='center', fontsize=12)
                axes[sample_idx, col_idx].set_title(f"CloneFinder: Sample {sample}", fontsize=12)
                col_idx += 1
                continue
                
            axes[sample_idx, col_idx].bar(sample_clonefinder.index, sample_clonefinder.values)
            axes[sample_idx, col_idx].set_title(f"CloneFinder: Sample {sample}", fontsize=12)
            axes[sample_idx, col_idx].set_ylim(0, 1.0)
            plt.setp(axes[sample_idx, col_idx].get_xticklabels(), rotation=45, ha="right", 
                    rotation_mode="anchor", fontsize=10)
                
            # Only add y-label to the first column if it's the first tool
            if col_idx == 0:
                axes[sample_idx, col_idx].set_ylabel("Frequency", fontsize=12)
                
            col_idx += 1
            
        # LICHeE plot - same fix pattern
        if lichee_data is not None:
            if sample in lichee_data.index:
                sample_lichee = lichee_data.loc[sample]
            elif sample.isdigit() and int(sample) in lichee_data.index:
                sample_lichee = lichee_data.loc[int(sample)]
            else:
                axes[sample_idx, col_idx].text(0.5, 0.5, "No LICHeE Data", 
                                            ha='center', va='center', fontsize=12)
                axes[sample_idx, col_idx].set_title(f"LICHeE: Sample {sample}", fontsize=12)
                col_idx += 1
                continue
                
            axes[sample_idx, col_idx].bar(sample_lichee.index, sample_lichee.values)
            axes[sample_idx, col_idx].set_title(f"LICHeE: Sample {sample}", fontsize=12)
            axes[sample_idx, col_idx].set_ylim(0, 1.0)
            plt.setp(axes[sample_idx, col_idx].get_xticklabels(), rotation=45, ha="right", 
                    rotation_mode="anchor", fontsize=10)
                
            # Only add y-label to the first column if it's the first tool
            if col_idx == 0:
                axes[sample_idx, col_idx].set_ylabel("Frequency", fontsize=12)
    
    # Add a main title for the entire figure
    fig.suptitle(f"Clonal Composition Comparison - Patient {patient_id}", fontsize=16, y=0.99)
    
    # Adjust layout for better spacing
    plt.tight_layout(rect=[0, 0, 1, 0.98])  # Leave room for suptitle
    
    # Save the combined figure
    save_path = os.path.join(output_dir, f"{patient_id}_all_samples_bar_graphs.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved combined bar graphs for all samples to: {save_path}")
    
    return save_path

def plot_clone_heatmap(data, title, cmap='coolwarm', figsize=(10, 8), save_path=None):
    """
    Plot a heatmap visualization of clone frequencies
    
    Args:
        data (pandas.DataFrame): DataFrame with samples as rows and clones as columns
        title (str): Title for the heatmap
        cmap (str): Colormap to use (default: 'coolwarm' - blue for low, red for high)
        figsize (tuple): Figure size (width, height)
        save_path (str, optional): Path to save the figure
    
    Returns:
        matplotlib.figure.Figure: The heatmap figure
    """
    print(data)
    # Ensure data is numeric
    data_numeric = data.copy()
    # Convert to float - this will help identify any non-numeric values
    for col in data_numeric.columns:
        data_numeric[col] = pd.to_numeric(data_numeric[col], errors='coerce')
    
    # Create figure
    fig = plt.figure(figsize=figsize)
    
    # Create heatmap with numeric data - using coolwarm colormap (blue for low, red for high)
    ax = sns.heatmap(data_numeric, annot=True, cmap=cmap, 
                     vmin=0, vmax=1, 
                     fmt='.2f', linewidths=.5)
    
    # Set title and labels
    plt.title(title, fontsize=14)
    plt.ylabel('Samples', fontsize=12)
    plt.xlabel('Clones', fontsize=12)
    
    # Rotate y-axis labels
    plt.yticks(rotation=0)
    
    # Adjust layout
    plt.tight_layout()
    
    # Save figure if path is provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved figure to: {save_path}")
    
    return fig

def visualize_clonality_for_patient(patient_id, results_dir="results"):
    """
    Visualize PyClone-VI, CloneFinder, and LICHeE results for a patient and save the heatmaps
    and sample-specific bar graphs directly in the patient's results directory
    
    Args:
        patient_id (str): Patient identifier
        results_dir (str): Base directory for results
    
    Returns:
        dict: Dictionary with paths to generated figures
    """
    # Get patient directory
    patient_dir = os.path.join(results_dir, patient_id)
    
    if not os.path.isdir(patient_dir):
        print(f"Error: Patient directory '{patient_dir}' not found.")
        return {}
    
    outputs = {}
    pyclone_data = None
    clonefinder_data = None
    lichee_data = None
    
    # Check for and process PyClone-VI results
    pyclone_file = os.path.join(patient_dir, f"{patient_id}_pyclone_results.tsv")
    if os.path.isfile(pyclone_file):
        print(f"Processing PyClone-VI results for patient {patient_id}...")
        
        # Parse results using our fixed function
        pyclone_data = parse_pyclone_results(pyclone_file)
        
        # Create heatmap visualization and save directly in patient directory
        save_path = os.path.join(patient_dir, f"{patient_id}_pyclone_heatmap.png")
        fig = plot_clone_heatmap(pyclone_data, 
                               title=f"PyClone-VI Clonal Composition - Patient {patient_id}",
                               save_path=save_path)
        plt.close(fig)  # Close figure to avoid display in notebook
        
        outputs['pyclone_heatmap'] = save_path
        
        print(f"PyClone-VI heatmap saved to: {save_path}")
    else:
        print(f"Warning: PyClone-VI results file '{pyclone_file}' not found.")
        
    # Check for and process CloneFinder results
    clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_input_CloneFinder.txt")
    if os.path.isfile(clonefinder_file):
        print(f"Processing CloneFinder results for patient {patient_id}...")
        
        # Parse results
        clonefinder_data = parse_clonefinder_results(clonefinder_file)
        
        # Create heatmap visualization and save directly in patient directory
        save_path = os.path.join(patient_dir, f"{patient_id}_clonefinder_heatmap.png")
        fig = plot_clone_heatmap(clonefinder_data, 
                               title=f"CloneFinder Clonal Composition - Patient {patient_id}",
                               save_path=save_path)
        plt.close(fig)  # Close figure to avoid display in notebook
        
        outputs['clonefinder_heatmap'] = save_path
        
        print(f"CloneFinder heatmap saved to: {save_path}")
    else:
        # Try alternative file naming patterns
        alt_pattern = os.path.join(patient_dir, f"{patient_id}_clonefinder_inputsnv_CloneFinder.txt")
        if os.path.isfile(alt_pattern):
            print(f"Processing CloneFinder results for patient {patient_id} (using alternative file)...")
            
            # Parse results
            clonefinder_data = parse_clonefinder_results(alt_pattern)
            
            # Create heatmap visualization and save directly in patient directory
            save_path = os.path.join(patient_dir, f"{patient_id}_clonefinder_heatmap.png")
            fig = plot_clone_heatmap(clonefinder_data, 
                                   title=f"CloneFinder Clonal Composition - Patient {patient_id}",
                                   save_path=save_path)
            plt.close(fig)  # Close figure to avoid display in notebook
            
            outputs['clonefinder_heatmap'] = save_path
            
            print(f"CloneFinder heatmap saved to: {save_path}")
        else:
            print(f"Warning: CloneFinder results file not found for patient {patient_id}.")
    
    # Check for and process LICHeE results
    lichee_file = os.path.join(patient_dir, f"{patient_id}_lichee_input.txt.trees.txt")
    if os.path.isfile(lichee_file):
        print(f"Processing LICHeE results for patient {patient_id}...")
        
        # Parse results
        lichee_data = parse_lichee_results(lichee_file)
        
        if not lichee_data.empty:
            # Create heatmap visualization and save directly in patient directory
            save_path = os.path.join(patient_dir, f"{patient_id}_lichee_heatmap.png")
            fig = plot_clone_heatmap(lichee_data, 
                                   title=f"LICHeE Clonal Composition - Patient {patient_id}",
                                   save_path=save_path)
            plt.close(fig)  # Close figure to avoid display in notebook
            
            outputs['lichee_heatmap'] = save_path
            
            print(f"LICHeE heatmap saved to: {save_path}")
    else:
        # Try alternative file naming patterns
        alt_patterns = [
            os.path.join(patient_dir, f"{patient_id}.lichee.output.txt"),
            os.path.join(patient_dir, f"{patient_id}_lichee_output.txt"),
            os.path.join(patient_dir, f"{patient_id}_trees.txt")
        ]
        
        found_file = False
        for alt_pattern in alt_patterns:
            if os.path.isfile(alt_pattern):
                print(f"Processing LICHeE results for patient {patient_id} (using file: {alt_pattern})...")
                
                # Parse results
                lichee_data = parse_lichee_results(alt_pattern)
                
                if not lichee_data.empty:
                    # Create heatmap visualization and save directly in patient directory
                    save_path = os.path.join(patient_dir, f"{patient_id}_lichee_heatmap.png")
                    fig = plot_clone_heatmap(lichee_data, 
                                           title=f"LICHeE Clonal Composition - Patient {patient_id}",
                                           save_path=save_path)
                    plt.close(fig)  # Close figure to avoid display in notebook
                    
                    outputs['lichee_heatmap'] = save_path
                    
                    print(f"LICHeE heatmap saved to: {save_path}")
                    found_file = True
                    break
        
        if not found_file:
            print(f"Warning: LICHeE results file not found for patient {patient_id}.")
    
    # Create sample-specific bar graphs if we have data from at least one tool
    data_sources = [d for d in [pyclone_data, clonefinder_data, lichee_data] if d is not None]
    if len(data_sources) >= 1:
        print(f"Creating combined bar graphs for patient {patient_id}...")
        bar_graph_file = plot_sample_bar_graphs(pyclone_data, clonefinder_data, lichee_data,
                                             patient_id, patient_dir)
        outputs['sample_bar_graphs'] = bar_graph_file
    
    return outputs

def visualize_all_patients(results_dir="results"):
    """
    Visualize PyClone-VI, CloneFinder, and LICHeE results for all patients and save
    directly in each patient's directory
    
    Args:
        results_dir (str): Base directory for results
        
    Returns:
        dict: Dictionary with paths to generated figures for each patient
    """
    # Find all patient directories
    patient_dirs = [d for d in os.listdir(results_dir) 
                   if os.path.isdir(os.path.join(results_dir, d))]
    
    if not patient_dirs:
        print(f"No patient directories found in {results_dir}")
        return {}
    
    results = {}
    
    for patient_id in patient_dirs:
        outputs = visualize_clonality_for_patient(patient_id, results_dir)
        if outputs:
            results[patient_id] = outputs
    
    return results

In [417]:
import os

def get_all_patient_ids(results_dir="results"):
    """
    Get all patient IDs by listing subdirectories under the results directory.
    
    Args:
        results_dir (str): Path to the base results directory
        
    Returns:
        list: List of patient IDs (directory names)
    """
    # Check if the results directory exists
    if not os.path.isdir(results_dir):
        print(f"Warning: Results directory '{results_dir}' not found.")
        return []
    
    # Get all directories (potential patient IDs)
    patient_ids = []
    for item in os.listdir(results_dir):
        full_path = os.path.join(results_dir, item)
        # Only include directories (not files)
        if os.path.isdir(full_path):
            patient_ids.append(item)
    
    print(f"Found {len(patient_ids)} patient directories: {', '.join(patient_ids)}")
    return sorted(patient_ids)


In [434]:
# patient_ids = get_all_patient_ids() 

# for patient_id in patient_ids:
#     visualize_clonality_for_patient(patient_id, results_dir="results")

visualize_clonality_for_patient("G12", results_dir="results")

Processing PyClone-VI results for patient G12...
Parsing PyClone-VI results from: results/G12/G12_pyclone_results.tsv
Found 4 samples and 13 clusters
    Clone 0   Clone 1   Clone 2   Clone 3  Clone 4   Clone 5   Clone 6  \
B  0.093872  0.000000  0.171427  0.005709  0.00000  0.000000  0.184994   
C  0.082438  0.165917  0.152502  0.000000  0.00000  0.161551  0.070769   
E  0.089788  0.000000  0.139395  0.006949  0.00000  0.156406  0.000000   
F  0.085420  0.054390  0.192806  0.000000  0.03228  0.113197  0.062903   

    Clone 7   Clone 8   Clone 9  Clone 10  Clone 11  Clone 12  
B  0.000000  0.188423  0.029130  0.000000  0.172633  0.153810  
C  0.000000  0.167898  0.031733  0.011854  0.155322  0.000017  
E  0.093864  0.172007  0.039906  0.000000  0.142474  0.159210  
F  0.002528  0.278604  0.000436  0.000000  0.072985  0.104451  


/var/folders/ct/zks035_n5vn9xb6d7n4xk5200000gn/T/ipykernel_21952/272564227.py:111: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  freq_matrix = freq_matrix.fillna(0.0)


Saved figure to: results/G12/G12_pyclone_heatmap.png
PyClone-VI heatmap saved to: results/G12/G12_pyclone_heatmap.png
Processing CloneFinder results for patient G12 (using alternative file)...
Parsing CloneFinder results from: results/G12/G12_clonefinder_inputsnv_CloneFinder.txt
          Clone1   Clone2   Clone3    Clone4    Clone5    Clone6    Clone7
Sample                                                                    
B       0.591059  0.35222  0.00000  0.000000  0.000000  0.000000  0.000000
C       0.000000  0.00000  0.73277  0.217834  0.000000  0.000000  0.000000
E       0.726283  0.00000  0.00000  0.000000  0.129531  0.000000  0.000000
F       0.000000  0.00000  0.00000  0.000000  0.000000  0.518508  0.134874
Saved figure to: results/G12/G12_clonefinder_heatmap.png
CloneFinder heatmap saved to: results/G12/G12_clonefinder_heatmap.png
Processing LICHeE results for patient G12...
Parsing LICHeE results from: results/G12/G12_lichee_input.txt.trees.txt
  Found 10 nodes
  Found 1

{'pyclone_heatmap': 'results/G12/G12_pyclone_heatmap.png',
 'clonefinder_heatmap': 'results/G12/G12_clonefinder_heatmap.png',
 'lichee_heatmap': 'results/G12/G12_lichee_heatmap.png',
 'sample_bar_graphs': 'results/G12/G12_all_samples_bar_graphs.png'}

In [412]:
# import os
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import subprocess
# import tempfile
# import sys

# def prepare_pyclone_for_clonevol(pyclone_file, output_dir):
#     """
#     Prepare PyClone-VI results for ClonEvol input,
#     ensuring cluster IDs are consecutive integers starting at 1
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results TSV file
#         output_dir (str): Directory for output files
        
#     Returns:
#         str: Path to the prepared ClonEvol input file
#     """
#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)
    
#     try:
#         # Load PyClone results
#         df = pd.read_csv(pyclone_file, sep='\t')
        
#         # Create a mapping of original cluster IDs to consecutive integers starting at 1
#         original_clusters = sorted(df['cluster_id'].unique())
#         cluster_mapping = {original: new+1 for new, original in enumerate(original_clusters)}
        
#         print(f"Remapping cluster IDs: {cluster_mapping}")
        
#         # Apply the mapping to create new consecutive cluster IDs
#         df['remapped_cluster_id'] = df['cluster_id'].map(cluster_mapping)
        
#         # Extract necessary columns for ClonEvol
#         clonevol_df = df[['remapped_cluster_id', 'sample_id', 'cellular_prevalence']]
        
#         # Rename columns to match ClonEvol expected format
#         clonevol_df = clonevol_df.rename(columns={
#             'remapped_cluster_id': 'cluster',
#             'sample_id': 'sample',
#             'cellular_prevalence': 'vaf'
#         })
        
#         # Sort by cluster and sample
#         clonevol_df = clonevol_df.sort_values(['cluster', 'sample'])
        
#         # Write to file
#         clonevol_input = os.path.join(output_dir, 'pyclone_for_clonevol.tsv')
#         clonevol_df.to_csv(clonevol_input, sep='\t', index=False)
        
#         # Save the mapping for reference
#         mapping_file = os.path.join(output_dir, 'cluster_id_mapping.txt')
#         with open(mapping_file, 'w') as f:
#             f.write("Original_ID,Remapped_ID\n")
#             for orig, new in cluster_mapping.items():
#                 f.write(f"{orig},{new}\n")
        
#         print(f"Prepared ClonEvol input file: {clonevol_input}")
#         print(f"Saved cluster ID mapping to: {mapping_file}")
#         return clonevol_input
    
#     except Exception as e:
#         print(f"Error preparing PyClone data for ClonEvol: {str(e)}")
#         return None

# def check_r_installation():
#     """Check if R is installed and available"""
#     try:
#         result = subprocess.run(["Rscript", "--version"], 
#                                stdout=subprocess.PIPE, 
#                                stderr=subprocess.PIPE,
#                                text=True)
#         if result.returncode == 0:
#             print(f"R is installed: {result.stdout or result.stderr}")
#             return True
#         else:
#             print("R does not appear to be installed or is not in the PATH")
#             return False
#     except Exception as e:
#         print(f"Error checking R installation: {str(e)}")
#         return False

# def check_clonevol_installation():
#     """Check if the ClonEvol package is installed in R"""
#     try:
#         # Create a temporary R script
#         with tempfile.NamedTemporaryFile(suffix='.R', mode='w', delete=False) as f:
#             f.write('if("clonevol" %in% rownames(installed.packages())) { cat("ClonEvol is installed") } else { cat("ClonEvol is NOT installed") }')
#             temp_script = f.name
        
#         # Run the script
#         result = subprocess.run(["Rscript", temp_script], 
#                                stdout=subprocess.PIPE, 
#                                stderr=subprocess.PIPE,
#                                text=True)
        
#         # Clean up
#         os.unlink(temp_script)
        
#         if "ClonEvol is installed" in result.stdout:
#             print("ClonEvol R package is installed")
#             return True
#         else:
#             print("ClonEvol R package is NOT installed")
#             print("To install ClonEvol in R, run:")
#             print("install.packages('devtools')")
#             print("devtools::install_github('hdng/clonevol')")
#             return False
    
#     except Exception as e:
#         print(f"Error checking ClonEvol installation: {str(e)}")
#         return False

# def run_clonevol(clonevol_input, output_dir, patient_id):
#     """
#     Run ClonEvol on the prepared input file using an R script
    
#     Args:
#         clonevol_input (str): Path to ClonEvol input file
#         output_dir (str): Directory for output files
#         patient_id (str): Patient identifier
        
#     Returns:
#         str: Path to the ClonEvol output file
#     """
#     # Check prerequisites
#     if not check_r_installation():
#         print("R is required to run ClonEvol")
#         return None
    
#     if not check_clonevol_installation():
#         print("ClonEvol R package is required")
#         return None
    
#     # Create R script path
#     r_script_path = os.path.join(output_dir, f"run_clonevol_{patient_id}.R")
#     output_file = os.path.join(output_dir, 'clonevol_clone_frequencies.tsv')
    
#     # Write R script content with better error handling
#     with open(r_script_path, 'w') as f:
#         f.write(f"""
# # Load required libraries with error handling
# tryCatch({{
#     library(clonevol)
#     library(ggplot2)
# }}, error = function(e) {{
#     cat("Error loading required libraries:", conditionMessage(e), "\\n")
#     quit(status = 1)
# }})

# # Set error handler to write to log
# error_log <- file("{os.path.join(output_dir, 'clonevol_error.log')}", "w")
# options(error = function() {{
#     cat(geterrmessage(), file = error_log)
#     close(error_log)
#     quit(status = 1)
# }})

# # Read input data
# cat("Reading input data...\\n")
# vafs <- tryCatch({{
#     read.table("{clonevol_input}", header=TRUE, sep="\\t")
# }}, error = function(e) {{
#     cat("Error reading input file:", conditionMessage(e), "\\n")
#     stop(e)
# }})

# # Convert to ClonEvol format
# cat("Converting to ClonEvol format...\\n")
# samples <- unique(vafs$sample)
# clusters <- unique(vafs$cluster)

# cat("Found", length(samples), "samples and", length(clusters), "clusters\\n")

# # Prepare cluster data frame
# cluster.data <- data.frame()
# for (cluster in clusters) {{
#     cluster.vafs <- vafs[vafs$cluster == cluster, ]
#     row <- c(cluster)
#     for (sample in samples) {{
#         sample.vaf <- cluster.vafs[cluster.vafs$sample == sample, "vaf"]
#         if (length(sample.vaf) > 0) {{
#             row <- c(row, sample.vaf[1])
#         }} else {{
#             row <- c(row, 0)
#         }}
#     }}
#     if (nrow(cluster.data) == 0) {{
#         cluster.data <- as.data.frame(t(row))
#     }} else {{
#         cluster.data <- rbind(cluster.data, row)
#     }}
# }}

# # Set column names
# colnames(cluster.data) <- c("cluster", samples)

# # Convert to numeric
# for (col in 2:ncol(cluster.data)) {{
#     cluster.data[,col] <- as.numeric(as.character(cluster.data[,col]))
# }}

# cat("Cluster data prepared. Building phylogeny models...\\n")

# # Build phylogeny models
# models <- list()
# try_model_sizes <- TRUE
# i <- 2

# while(try_model_sizes && i <= length(clusters)) {{
#     cat("Trying model with", i, "clones...\\n")
#     tryCatch({{
#         # Try models with i clones
#         model <- infer.clonal.models(variants=cluster.data, cluster.col.name="cluster",
#                               vaf.col.names=samples, subclonal.test="bootstrap",
#                               subclonal.test.model="non-parametric", num.boots=1000,
#                               founding.cluster=1,
#                               cluster.center="mean", ignore.clusters=NULL,
#                               min.cluster.vaf=0.05, sum.p=0.05, alpha=0.05)
        
#         # Store successful model
#         if (!is.null(model)) {{
#             models[[i-1]] <- model
#             cat("  - Model with", i, "clones successful\\n")
#         }}
#     }}, error = function(e) {{
#         cat("  - Model with", i, "clones failed:", conditionMessage(e), "\\n")
#         if (grepl("maximum number of iterations reached", conditionMessage(e)) || 
#             grepl("singular gradient", conditionMessage(e))) {{
#             # These are common errors when model is too complex
#             try_model_sizes <<- FALSE
#             cat("  - Stopping model search due to convergence issues\\n")
#         }}
#     }})
    
#     i <- i + 1
# }}

# # Find the best model
# best.model.idx <- 0
# if (length(models) > 0) {{
#     best.model.idx <- which.max(sapply(models, function(x) {{
#         if (is.null(x)) return(0)
#         return(length(x))
#     }}))
# }}

# if (length(best.model.idx) > 0 && best.model.idx > 0 && !is.null(models[[best.model.idx]])) {{
#     cat("Found best model with", best.model.idx+1, "clones\\n")
#     best.model <- models[[best.model.idx]]
    
#     # Create tree directory
#     tree_dir <- "{os.path.join(output_dir, 'clonevol_trees')}"
#     dir.create(tree_dir, showWarnings = FALSE, recursive = TRUE)
    
#     # Convert model to proper clone frequencies
#     cat("Converting model to clone frequencies...\\n")
#     tryCatch({{
#         # This will calculate clone frequencies based on evolutionary relationships
#         clone.data <- convert.consensus.tree.clone.to.branch(cluster.data, best.model)
        
#         # Write results to file
#         write.table(clone.data, file="{output_file}",
#                     sep="\\t", quote=FALSE, row.names=FALSE)
        
#         cat("Plotting results...\\n")
#         # Plot clone frequencies
#         pdf(file.path(tree_dir, "clonevol_clone_plot.pdf"))
#         plot.clonal.models(best.model)
#         dev.off()
        
#         # Plot tree
#         pdf(file.path(tree_dir, "clonevol_tree.pdf"))
#         plot.clones.as.fishplot(clone.data)
#         dev.off()
        
#         cat("ClonEvol analysis completed successfully\\n")
#     }}, error = function(e) {{
#         cat("Error generating clone frequencies:", conditionMessage(e), "\\n")
#         # Create a fallback simple clone frequency file using cluster.data
#         # This at least gives us some output even if the phylogenetic analysis fails
#         write.table(cluster.data, file="{os.path.join(output_dir, 'clonevol_simple_frequencies.tsv')}",
#                    sep="\\t", quote=FALSE, row.names=FALSE)
#         cat("Created fallback frequency file based on input clusters\\n")
#     }})
# }} else {{
#     cat("No valid models could be built\\n")
#     # Create a fallback simple frequency file
#     write.table(cluster.data, file="{os.path.join(output_dir, 'clonevol_simple_frequencies.tsv')}",
#                sep="\\t", quote=FALSE, row.names=FALSE)
#     cat("Created fallback frequency file based on input clusters\\n")
# }}
# """)
    
#     # Run R script with appropriate error handling
#     try:
#         print(f"Running ClonEvol R script: {r_script_path}")
#         result = subprocess.run(["Rscript", r_script_path], 
#                               stdout=subprocess.PIPE, 
#                               stderr=subprocess.PIPE,
#                               text=True,
#                               check=False)  # Don't raise exception on non-zero return
        
#         print("=== ClonEvol Output ===")
#         print(result.stdout)
        
#         if result.stderr:
#             print("=== ClonEvol Errors/Warnings ===")
#             print(result.stderr)
        
#         # Check if main output file exists
#         if os.path.exists(output_file):
#             print(f"ClonEvol generated clone frequencies file: {output_file}")
#             return output_file
        
#         # Check if fallback file exists
#         fallback_file = os.path.join(output_dir, 'clonevol_simple_frequencies.tsv')
#         if os.path.exists(fallback_file):
#             print(f"Using fallback clone frequencies: {fallback_file}")
#             return fallback_file
        
#         print("ClonEvol failed to generate any output files")
#         return None
        
#     except Exception as e:
#         print(f"Error running ClonEvol: {str(e)}")
#         return None

# def parse_clonevol_results(clonevol_output):
#     """
#     Parse ClonEvol results into a DataFrame with proper clone frequencies
    
#     Args:
#         clonevol_output (str): Path to ClonEvol output file
    
#     Returns:
#         pandas.DataFrame: DataFrame with samples as rows and clones as columns
#     """
#     if not os.path.exists(clonevol_output):
#         print(f"Error: ClonEvol output file not found: {clonevol_output}")
#         return None
    
#     try:
#         # Read ClonEvol results
#         df = pd.read_csv(clonevol_output, sep='\t')
        
#         # Extract sample columns (all except 'cluster')
#         samples = [col for col in df.columns if col != 'cluster']
        
#         # Check if columns are actually numeric
#         for col in samples:
#             if not pd.api.types.is_numeric_dtype(df[col]):
#                 print(f"Converting column {col} to numeric")
#                 df[col] = pd.to_numeric(df[col], errors='coerce')
        
#         # Pivot data to have samples as rows and clones as columns
#         clone_freq_df = pd.DataFrame(index=samples)
        
#         for _, row in df.iterrows():
#             clone_id = f"Clone {int(float(row['cluster']))}"
#             for sample in samples:
#                 clone_freq_df.loc[sample, clone_id] = float(row[sample])
        
#         # Fill NaN values with 0
#         clone_freq_df = clone_freq_df.fillna(0)
        
#         # Verify that rows sum to approximately 1
#         row_sums = clone_freq_df.sum(axis=1)
#         print("\nVerifying that clone frequencies sum to 1.0:")
#         for sample, row_sum in row_sums.items():
#             print(f"Sample {sample}: {row_sum:.4f}")
            
#             # If rows don't sum to approximately 1, normalize them
#             if not 0.95 <= row_sum <= 1.05:
#                 print(f"  Warning: Frequencies for {sample} don't sum to 1.0, normalizing...")
#                 clone_freq_df.loc[sample] = clone_freq_df.loc[sample] / row_sum
        
#         return clone_freq_df
    
#     except Exception as e:
#         print(f"Error parsing ClonEvol results: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

# def plot_clonevol_heatmap(clone_freq_df, patient_id, output_dir='.'):
#     """
#     Create a heatmap from ClonEvol clone frequencies
    
#     Args:
#         clone_freq_df (pandas.DataFrame): DataFrame with samples as rows and clones as columns
#         patient_id (str): Patient identifier for title
#         output_dir (str): Directory to save the output
        
#     Returns:
#         str: Path to the saved heatmap
#     """
#     if clone_freq_df is None or clone_freq_df.empty:
#         print("No data to plot")
#         return None
    
#     try:
#         # Create figure
#         plt.figure(figsize=(10, 6))
        
#         # Create heatmap with normalized data
#         ax = sns.heatmap(clone_freq_df, annot=True, cmap='coolwarm', 
#                          vmin=0, vmax=1, fmt='.2f', linewidths=.5)
        
#         # Set title and labels
#         plt.title(f"ClonEvol Evolutionary Clone Frequencies - Patient {patient_id}", fontsize=14)
#         plt.ylabel('Samples', fontsize=12)
#         plt.xlabel('Clones', fontsize=12)
        
#         # Adjust layout
#         plt.tight_layout()
        
#         # Save figure
#         save_path = os.path.join(output_dir, f"{patient_id}_clonevol_heatmap.png")
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')
#         plt.close()
#         print(f"Saved ClonEvol heatmap to: {save_path}")
        
#         return save_path
#     except Exception as e:
#         print(f"Error creating heatmap: {str(e)}")
#         return None

# def normalize_pyclone_results(pyclone_file):
#     """
#     Simple fallback method to normalize PyClone-VI results if ClonEvol fails
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results file
        
#     Returns:
#         pandas.DataFrame: DataFrame with normalized clone frequencies
#     """
#     try:
#         # Load PyClone results
#         df = pd.read_csv(pyclone_file, sep='\t')
        
#         # Get unique samples and clusters
#         samples = df['sample_id'].unique()
#         clusters = sorted(df['cluster_id'].unique())
        
#         print(f"Found {len(samples)} samples and {len(clusters)} clusters")
        
#         # Create a temporary dictionary to hold values
#         temp_data = {}
        
#         # Process each sample
#         for sample in samples:
#             temp_data[sample] = {}
#             sample_df = df[df['sample_id'] == sample]
            
#             # Calculate average cellular prevalence for each cluster
#             for cluster in clusters:
#                 cluster_data = sample_df[sample_df['cluster_id'] == cluster]
#                 if not cluster_data.empty:
#                     temp_data[sample][f'Clone {cluster}'] = float(cluster_data['cellular_prevalence'].mean())
#                 else:
#                     temp_data[sample][f'Clone {cluster}'] = 0.0
        
#         # Convert to DataFrame
#         result_df = pd.DataFrame.from_dict(temp_data, orient='index')
        
#         # Normalize each row to sum to 1.0
#         for sample in result_df.index:
#             row_sum = result_df.loc[sample].sum()
#             if row_sum > 0:  # Avoid division by zero
#                 result_df.loc[sample] = result_df.loc[sample] / row_sum
        
#         # Verify row sums
#         for sample in result_df.index:
#             print(f"Sample {sample} sum: {result_df.loc[sample].sum():.4f}")
        
#         return result_df
#     except Exception as e:
#         print(f"Error normalizing PyClone results: {str(e)}")
#         return None

# def run_full_pipeline(pyclone_file, patient_id, output_dir='.'):
#     """
#     Run the full pipeline from PyClone-VI results to clone frequency heatmap
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results TSV file
#         patient_id (str): Patient identifier
#         output_dir (str): Directory for output files
        
#     Returns:
#         str: Path to the saved heatmap
#     """
#     print(f"\n{'='*80}\nProcessing patient {patient_id}\n{'='*80}")
    
#     # Create patient-specific output directory
#     patient_dir = os.path.join(output_dir, patient_id)
#     os.makedirs(patient_dir, exist_ok=True)
    
#     try:
#         # Try ClonEvol approach first
#         print("\n[1/5] Preparing PyClone data for ClonEvol...")
#         clonevol_input = prepare_pyclone_for_clonevol(pyclone_file, patient_dir)
        
#         if clonevol_input:
#             print("\n[2/5] Running ClonEvol...")
#             clonevol_output = run_clonevol(clonevol_input, patient_dir, patient_id)
            
#             if clonevol_output:
#                 print("\n[3/5] Parsing ClonEvol results...")
#                 clone_freq_df = parse_clonevol_results(clonevol_output)
                
#                 if clone_freq_df is not None:
#                     print("\n[4/5] Creating ClonEvol heatmap...")
#                     heatmap_path = plot_clonevol_heatmap(clone_freq_df, patient_id, patient_dir)
                    
#                     if heatmap_path:
#                         print("\n[5/5] Pipeline completed successfully!")
#                         return heatmap_path
        
#         # If any step failed, fall back to simple normalization
#         print("\nClonEvol approach failed. Using simple normalization as fallback.")
        
#         print("\n[1/3] Normalizing PyClone results...")
#         normalized_df = normalize_pyclone_results(pyclone_file)
        
#         if normalized_df is not None:
#             print("\n[2/3] Creating normalized heatmap...")
#             # Create figure
#             plt.figure(figsize=(10, 6))
            
#             # Create heatmap with normalized data
#             sns.heatmap(normalized_df, annot=True, cmap='coolwarm', 
#                         vmin=0, vmax=1, fmt='.2f', linewidths=.5)
            
#             # Set title and labels
#             plt.title(f"Normalized PyClone-VI Clonal Composition - Patient {patient_id}", fontsize=14)
#             plt.ylabel('Samples', fontsize=12)
#             plt.xlabel('Clones', fontsize=12)
            
#             # Adjust layout
#             plt.tight_layout()
            
#             # Save figure
#             save_path = os.path.join(patient_dir, f"{patient_id}_normalized_pyclone_heatmap.png")
#             plt.savefig(save_path, dpi=300, bbox_inches='tight')
#             plt.close()
            
#             print(f"\n[3/3] Saved normalized heatmap to: {save_path}")
#             return save_path
    
#     except Exception as e:
#         print(f"Error in pipeline: {str(e)}")
#         import traceback
#         traceback.print_exc()
    
#     print("All approaches failed. Unable to generate clone frequency heatmap.")
#     return None

# patient_ids = get_all_patient_ids() 

# for patient_id in patient_ids:
#     run_full_pipeline(f"results/{patient_id}/{patient_id}_pyclone_results.tsv", patient_id, "results")

In [411]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# def convert_pyclone_to_clone_frequencies(pyclone_file, use_maximum=True):
#     """
#     Convert PyClone-VI results to clone frequencies by:
#     1. Grouping mutations by cluster and sample
#     2. Taking maximum (or median) cellular prevalence as proxy for clonal frequency
#     3. Normalizing to ensure frequencies sum to 1.0 per sample
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results TSV file
#         use_maximum (bool): If True, use maximum cellular prevalence; if False, use median
    
#     Returns:
#         pandas.DataFrame: DataFrame with samples as rows and clones as columns
#     """
#     print(f"Processing PyClone-VI results from: {pyclone_file}")
    
#     try:
#         # Load PyClone results
#         df = pd.read_csv(pyclone_file, sep='\t')
        
#         # Get unique samples and clusters
#         samples = sorted(df['sample_id'].unique())
#         clusters = sorted(df['cluster_id'].unique())
        
#         print(f"Found {len(samples)} samples and {len(clusters)} clusters")
        
#         # Group by cluster_id and sample_id, and calculate representative value (max or median)
#         grouped = df.groupby(['cluster_id', 'sample_id'])
        
#         # Create a DataFrame to store the representative values
#         if use_maximum:
#             cluster_prev = grouped['cellular_prevalence'].max().reset_index()
#             print("Using MAXIMUM cellular prevalence as proxy for clonal frequency")
#         else:
#             cluster_prev = grouped['cellular_prevalence'].median().reset_index()
#             print("Using MEDIAN cellular prevalence as proxy for clonal frequency")
        
#         # Create a pivot table with samples as rows and clusters as columns
#         prev_matrix = pd.pivot_table(
#             cluster_prev,
#             values='cellular_prevalence',
#             index='sample_id',
#             columns='cluster_id',
#             fill_value=0
#         )
        
#         # Rename columns to 'Clone X'
#         prev_matrix.columns = [f'Clone {int(col)}' for col in prev_matrix.columns]
        
#         # Normalize rows to sum to 1.0
#         normalized_df = pd.DataFrame(index=prev_matrix.index, columns=prev_matrix.columns)
        
#         for sample in prev_matrix.index:
#             row_sum = prev_matrix.loc[sample].sum()
#             if row_sum > 0:  # Avoid division by zero
#                 normalized_df.loc[sample] = prev_matrix.loc[sample] / row_sum
#             else:
#                 normalized_df.loc[sample] = prev_matrix.loc[sample]  # Keep zeros if sum is zero
        
#         # Verify that rows sum to 1.0
#         print("\nVerifying normalized frequencies sum to 1.0:")
#         for sample in normalized_df.index:
#             row_sum = normalized_df.loc[sample].sum()
#             print(f"Sample {sample}: {row_sum:.4f}")
        
#         return normalized_df
        
#     except Exception as e:
#         print(f"Error processing PyClone-VI results: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

# def plot_clone_frequency_heatmap(normalized_df, patient_id, output_dir='.', method='maximum'):
#     """
#     Create a heatmap visualization of the clone frequencies
    
#     Args:
#         normalized_df (pandas.DataFrame): DataFrame with normalized clone frequencies
#         patient_id (str): Patient identifier for title
#         output_dir (str): Directory to save output
#         method (str): Method used for frequency estimation ('maximum' or 'median')
        
#     Returns:
#         str: Path to saved heatmap
#     """
#     if normalized_df is None or normalized_df.empty:
#         print("No data to plot")
#         return None
    
#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)
    
#     try:
#         # Create figure
#         plt.figure(figsize=(10, 6))
        
#         # Plot heatmap
#         sns.heatmap(normalized_df, annot=True, cmap='Blues_r', 
#                    vmin=0, vmax=1, fmt='.2f', linewidths=.5)
        
#         # Set title and labels
#         method_str = "Maximum" if method == 'maximum' else "Median"
#         plt.title(f"PyClone-VI Clonal Composition - Patient {patient_id}\n({method_str} Prevalence Method)", fontsize=14)
#         plt.ylabel('Samples', fontsize=12)
#         plt.xlabel('Clones', fontsize=12)
        
#         # Adjust layout
#         plt.tight_layout()
        
#         # Save figure
#         save_path = os.path.join(output_dir, f"{patient_id}_pyclone_clonefreq_{method}.png")
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')
#         plt.close()
        
#         print(f"Saved clone frequency heatmap to: {save_path}")
#         return save_path
        
#     except Exception as e:
#         print(f"Error creating heatmap: {str(e)}")
#         return None

# def create_clone_frequency_heatmap(pyclone_file, patient_id, output_dir='.', method='maximum'):
#     """
#     Full pipeline to convert PyClone-VI results to clone frequencies and create heatmap
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results file
#         patient_id (str): Patient identifier
#         output_dir (str): Directory to save output
#         method (str): Method for frequency estimation ('maximum' or 'median')
        
#     Returns:
#         str: Path to saved heatmap
#     """
#     # Convert PyClone-VI results to clone frequencies
#     use_maximum = (method.lower() == 'maximum')
#     normalized_df = convert_pyclone_to_clone_frequencies(pyclone_file, use_maximum=use_maximum)
    
#     if normalized_df is not None:
#         # Create and save heatmap
#         return plot_clone_frequency_heatmap(normalized_df, patient_id, output_dir, method)
#     else:
#         print("Failed to generate clone frequencies")
#         return None

# def compare_estimation_methods(pyclone_file, patient_id, output_dir='.'):
#     """
#     Compare maximum and median methods for clone frequency estimation
    
#     Args:
#         pyclone_file (str): Path to PyClone-VI results file
#         patient_id (str): Patient identifier
#         output_dir (str): Directory to save output
        
#     Returns:
#         tuple: Paths to saved heatmaps (maximum, median)
#     """
#     print(f"\n{'='*80}\nProcessing patient {patient_id} with maximum method\n{'='*80}")
#     max_path = create_clone_frequency_heatmap(pyclone_file, patient_id, output_dir, method='maximum')
    
#     print(f"\n{'='*80}\nProcessing patient {patient_id} with median method\n{'='*80}")
#     med_path = create_clone_frequency_heatmap(pyclone_file, patient_id, output_dir, method='median')
    
#     return max_path, med_path

# # Example usage:
# compare_estimation_methods("results/G12/G12_pyclone_results.tsv", "G12", "results/G12")

## Compare results

In [ ]:
import os

def get_all_patient_ids(results_dir="results"):
    """
    Get all patient IDs by listing subdirectories under the results directory.
    
    Args:
        results_dir (str): Path to the base results directory
        
    Returns:
        list: List of patient IDs (directory names)
    """
    # Check if the results directory exists
    if not os.path.isdir(results_dir):
        print(f"Warning: Results directory '{results_dir}' not found.")
        return []
    
    # Get all directories (potential patient IDs)
    patient_ids = []
    for item in os.listdir(results_dir):
        full_path = os.path.join(results_dir, item)
        # Only include directories (not files)
        if os.path.isdir(full_path):
            patient_ids.append(item)
    
    print(f"Found {len(patient_ids)} patient directories: {', '.join(patient_ids)}")
    return sorted(patient_ids)

In [ ]:
# def calculate_diversity_metrics_per_sample(patient_ids, results_dir="results", save_to_file=True):
#     """
#     Calculate diversity metrics for each patient and sample combination using CloneFinder and LICHeE results.
#     Optionally saves results to a CSV file.
    
#     Args:
#         patient_ids (list): List of patient identifiers
#         results_dir (str): Base directory for results
#         save_to_file (bool): Whether to save metrics to a CSV file
    
#     Returns:
#         pandas.DataFrame: DataFrame with diversity metrics for each patient/sample combination
#         str: Path to saved CSV file (if save_to_file is True)
#     """
#     # Create a dataframe to store results
#     columns = ['Patient_ID', 'Sample_ID', 
#                'CloneFinder_Num_Clones', 'CloneFinder_Shannon_Index',
#                'LICHeE_Num_Clones', 'LICHeE_Shannon_Index']
#     results = pd.DataFrame(columns=columns)
    
#     for patient_id in patient_ids:
#         patient_dir = os.path.join(results_dir, patient_id)
        
#         if not os.path.isdir(patient_dir):
#             print(f"Warning: Patient directory '{patient_dir}' not found.")
#             continue
        
#         # Process CloneFinder results
#         clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_input_CloneFinder.txt")
#         if not os.path.isfile(clonefinder_file):
#             # Try alternative file naming pattern
#             clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_inputsnv_CloneFinder.txt")
        
#         clonefinder_data = None
#         if os.path.isfile(clonefinder_file):
#             try:
#                 # Parse CloneFinder results
#                 clonefinder_data = parse_clonefinder_results(clonefinder_file)
#             except Exception as e:
#                 print(f"Error processing CloneFinder data for patient {patient_id}: {e}")
        
#         # Process LICHeE results
#         lichee_file = os.path.join(patient_dir, f"{patient_id}_lichee_input.txt.trees.txt")
#         if not os.path.isfile(lichee_file):
#             # Try alternative file naming patterns
#             alt_patterns = [
#                 os.path.join(patient_dir, f"{patient_id}.lichee.output.txt"),
#                 os.path.join(patient_dir, f"{patient_id}_lichee_output.txt"),
#                 os.path.join(patient_dir, f"{patient_id}_trees.txt")
#             ]
            
#             for alt_pattern in alt_patterns:
#                 if os.path.isfile(alt_pattern):
#                     lichee_file = alt_pattern
#                     break
        
#         lichee_data = None
#         if os.path.isfile(lichee_file):
#             try:
#                 # Parse LICHeE results
#                 lichee_data = parse_lichee_results(lichee_file)
#             except Exception as e:
#                 print(f"Error processing LICHeE data for patient {patient_id}: {e}")
        
#         # Get all unique samples across both tools
#         all_samples = set()
#         if clonefinder_data is not None and not clonefinder_data.empty:
#             all_samples.update(clonefinder_data.index)
#         if lichee_data is not None and not lichee_data.empty:
#             all_samples.update(lichee_data.index)
        
#         # Calculate metrics for each sample
#         for original_sample in all_samples:
#             sample_metrics = {
#                 'Patient_ID': patient_id,
#                 'Sample_ID': original_sample,  # Keep original type
#                 'CloneFinder_Num_Clones': 0, 
#                 'CloneFinder_Shannon_Index': np.nan,
#                 'LICHeE_Num_Clones': 0, 
#                 'LICHeE_Shannon_Index': np.nan
#             }
            
#             # Process CloneFinder data for this sample
#             if clonefinder_data is not None and not clonefinder_data.empty:
#                 # Try to find the sample in different formats
#                 sample_found = False
#                 sample_to_use = None
                
#                 # Try direct match first
#                 if original_sample in clonefinder_data.index:
#                     sample_found = True
#                     sample_to_use = original_sample
#                 # Try converting string to int
#                 elif isinstance(original_sample, str) and original_sample.isdigit():
#                     int_sample = int(original_sample)
#                     if int_sample in clonefinder_data.index:
#                         sample_found = True
#                         sample_to_use = int_sample
#                 # Try converting int to string
#                 elif isinstance(original_sample, (int, float)):
#                     str_sample = str(int(original_sample))
#                     if str_sample in clonefinder_data.index:
#                         sample_found = True
#                         sample_to_use = str_sample
                
#                 if sample_found and sample_to_use is not None:
#                     # Get clone frequencies for this sample
#                     frequencies = clonefinder_data.loc[sample_to_use].values
#                     # Only consider non-zero frequencies
#                     cf_nonzero_freqs = frequencies[frequencies > 0.00001]
                    
#                     if len(cf_nonzero_freqs) > 0:
#                         # Count non-zero clones
#                         sample_metrics['CloneFinder_Num_Clones'] = len(cf_nonzero_freqs)
                        
#                         # Normalize frequencies to sum to 1
#                         normalized_freqs = cf_nonzero_freqs / np.sum(cf_nonzero_freqs)
                        
#                         # Calculate Shannon diversity index: -sum(p * ln(p))
#                         try:
#                             shannon_index = -np.sum(normalized_freqs * np.log(normalized_freqs))
#                             sample_metrics['CloneFinder_Shannon_Index'] = shannon_index
#                         except Exception as e:
#                             print(f"Error calculating CloneFinder Shannon Index: {e}")
            
#             # Process LICHeE data for this sample
#             if lichee_data is not None and not lichee_data.empty:
#                 # Try to find the sample in different formats
#                 sample_found = False
#                 sample_to_use = None
                
#                 # Try direct match first
#                 if original_sample in lichee_data.index:
#                     sample_found = True
#                     sample_to_use = original_sample
#                 # Try converting string to int
#                 elif isinstance(original_sample, str) and original_sample.isdigit():
#                     int_sample = int(original_sample)
#                     if int_sample in lichee_data.index:
#                         sample_found = True
#                         sample_to_use = int_sample
#                 # Try converting int to string
#                 elif isinstance(original_sample, (int, float)):
#                     str_sample = str(int(original_sample))
#                     if str_sample in lichee_data.index:
#                         sample_found = True
#                         sample_to_use = str_sample
                
#                 if sample_found and sample_to_use is not None:
#                     # Get clone frequencies for this sample
#                     frequencies = lichee_data.loc[sample_to_use].values
#                     # Only consider non-zero frequencies
#                     lich_nonzero_freqs = frequencies[frequencies > 0.00001]
                    
#                     if len(lich_nonzero_freqs) > 0:
#                         # Count non-zero clones
#                         sample_metrics['LICHeE_Num_Clones'] = len(lich_nonzero_freqs)
                        
#                         # Normalize frequencies to sum to 1
#                         normalized_freqs = lich_nonzero_freqs / np.sum(lich_nonzero_freqs)
                        
#                         # Calculate Shannon diversity index: -sum(p * ln(p))
#                         try:
#                             shannon_index = -np.sum(normalized_freqs * np.log(normalized_freqs))
#                             sample_metrics['LICHeE_Shannon_Index'] = shannon_index
#                         except Exception as e:
#                             print(f"Error calculating LICHeE Shannon Index: {e}")
            
#             # Add sample metrics to results dataframe
#             results = pd.concat([results, pd.DataFrame([sample_metrics])], ignore_index=True)
    
#     # Save results to file if requested
#     saved_file_path = None
#     if save_to_file and not results.empty:
#         if len(patient_ids) == 1:
#             # Single patient - save in patient directory
#             patient_id = patient_ids[0]
#             patient_dir = os.path.join(results_dir, patient_id)
#             saved_file_path = os.path.join(patient_dir, f"{patient_id}_diversity_metrics.csv")
#         else:
#             saved_file_path = os.path.join(results_dir, "all_patients_diversity_metrics.csv")
        
#         results.to_csv(saved_file_path, index=False)
#         print(f"Diversity metrics saved to: {saved_file_path}")
    
#     if save_to_file:
#         return results, saved_file_path
#     else:
#         return results

In [437]:
def calculate_diversity_metrics_per_sample(patient_ids, results_dir="results", save_to_file=True):
    """
    Calculate diversity metrics for each patient and sample combination using PyClone-VI, CloneFinder, and LICHeE results.
    Optionally saves results to a CSV file.
    
    Args:
        patient_ids (list): List of patient identifiers
        results_dir (str): Base directory for results
        save_to_file (bool): Whether to save metrics to a CSV file
    
    Returns:
        pandas.DataFrame: DataFrame with diversity metrics for each patient/sample combination
        str: Path to saved CSV file (if save_to_file is True)
    """
    # Create a dataframe to store results
    columns = ['Patient_ID', 'Sample_ID', 
               'PyClone_Num_Clones', 'PyClone_Shannon_Index',
               'CloneFinder_Num_Clones', 'CloneFinder_Shannon_Index',
               'LICHeE_Num_Clones', 'LICHeE_Shannon_Index']
    results = pd.DataFrame(columns=columns)
    
    for patient_id in patient_ids:
        patient_dir = os.path.join(results_dir, patient_id)
        
        if not os.path.isdir(patient_dir):
            print(f"Warning: Patient directory '{patient_dir}' not found.")
            continue
        
        # Process PyClone results
        pyclone_file = os.path.join(patient_dir, f"{patient_id}_pyclone_results.tsv")
        if not os.path.isfile(pyclone_file):
            # Try alternative file naming patterns
            alt_patterns = [
                os.path.join(patient_dir, f"{patient_id}.pyclone.output.tsv"),
                os.path.join(patient_dir, "pyclone_output.tsv"),
                os.path.join(patient_dir, "cluster_assignments.tsv")
            ]
            
            for alt_pattern in alt_patterns:
                if os.path.isfile(alt_pattern):
                    pyclone_file = alt_pattern
                    break
        
        pyclone_data = None
        if os.path.isfile(pyclone_file):
            try:
                # Parse PyClone results (using the modified function that only includes mutations with prevalence > 0)
                pyclone_data = parse_pyclone_results(pyclone_file)
            except Exception as e:
                print(f"Error processing PyClone data for patient {patient_id}: {e}")
        
        # Process CloneFinder results
        clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_input_CloneFinder.txt")
        if not os.path.isfile(clonefinder_file):
            # Try alternative file naming pattern
            clonefinder_file = os.path.join(patient_dir, f"{patient_id}_clonefinder_inputsnv_CloneFinder.txt")
        
        clonefinder_data = None
        if os.path.isfile(clonefinder_file):
            try:
                # Parse CloneFinder results
                clonefinder_data = parse_clonefinder_results(clonefinder_file)
            except Exception as e:
                print(f"Error processing CloneFinder data for patient {patient_id}: {e}")
        
        # Process LICHeE results
        lichee_file = os.path.join(patient_dir, f"{patient_id}_lichee_input.txt.trees.txt")
        if not os.path.isfile(lichee_file):
            # Try alternative file naming patterns
            alt_patterns = [
                os.path.join(patient_dir, f"{patient_id}.lichee.output.txt"),
                os.path.join(patient_dir, f"{patient_id}_lichee_output.txt"),
                os.path.join(patient_dir, f"{patient_id}_trees.txt")
            ]
            
            for alt_pattern in alt_patterns:
                if os.path.isfile(alt_pattern):
                    lichee_file = alt_pattern
                    break
        
        lichee_data = None
        if os.path.isfile(lichee_file):
            try:
                # Parse LICHeE results
                lichee_data = parse_lichee_results(lichee_file)
            except Exception as e:
                print(f"Error processing LICHeE data for patient {patient_id}: {e}")
        
        # Get all unique samples across all tools
        all_samples = set()
        if pyclone_data is not None and not pyclone_data.empty:
            all_samples.update(pyclone_data.index)
        if clonefinder_data is not None and not clonefinder_data.empty:
            all_samples.update(clonefinder_data.index)
        if lichee_data is not None and not lichee_data.empty:
            all_samples.update(lichee_data.index)
        
        # Calculate metrics for each sample
        for original_sample in all_samples:
            sample_metrics = {
                'Patient_ID': patient_id,
                'Sample_ID': original_sample,  # Keep original type
                'PyClone_Num_Clones': 0,
                'PyClone_Shannon_Index': np.nan,
                'CloneFinder_Num_Clones': 0, 
                'CloneFinder_Shannon_Index': np.nan,
                'LICHeE_Num_Clones': 0, 
                'LICHeE_Shannon_Index': np.nan
            }
            
            # Process PyClone data for this sample
            if pyclone_data is not None and not pyclone_data.empty:
                # Try to find the sample in different formats
                sample_found = False
                sample_to_use = None
                
                # Try direct match first
                if original_sample in pyclone_data.index:
                    sample_found = True
                    sample_to_use = original_sample
                # Try converting string to int
                elif isinstance(original_sample, str) and original_sample.isdigit():
                    int_sample = int(original_sample)
                    if int_sample in pyclone_data.index:
                        sample_found = True
                        sample_to_use = int_sample
                # Try converting int to string
                elif isinstance(original_sample, (int, float)):
                    str_sample = str(int(original_sample))
                    if str_sample in pyclone_data.index:
                        sample_found = True
                        sample_to_use = str_sample
                
                if sample_found and sample_to_use is not None:
                    # Get clone frequencies for this sample
                    frequencies = pyclone_data.loc[sample_to_use].values
                    # Only consider non-zero frequencies
                    pc_nonzero_freqs = frequencies[frequencies > 0.00001]
                    
                    if len(pc_nonzero_freqs) > 0:
                        # Count non-zero clones
                        sample_metrics['PyClone_Num_Clones'] = len(pc_nonzero_freqs)
                        
                        # Normalize frequencies to sum to 1
                        normalized_freqs = pc_nonzero_freqs / np.sum(pc_nonzero_freqs)
                        
                        # Calculate Shannon diversity index: -sum(p * ln(p))
                        try:
                            shannon_index = -np.sum(normalized_freqs * np.log(normalized_freqs))
                            sample_metrics['PyClone_Shannon_Index'] = shannon_index
                        except Exception as e:
                            print(f"Error calculating PyClone Shannon Index: {e}")
            
            # Process CloneFinder data for this sample
            if clonefinder_data is not None and not clonefinder_data.empty:
                # Try to find the sample in different formats
                sample_found = False
                sample_to_use = None
                
                # Try direct match first
                if original_sample in clonefinder_data.index:
                    sample_found = True
                    sample_to_use = original_sample
                # Try converting string to int
                elif isinstance(original_sample, str) and original_sample.isdigit():
                    int_sample = int(original_sample)
                    if int_sample in clonefinder_data.index:
                        sample_found = True
                        sample_to_use = int_sample
                # Try converting int to string
                elif isinstance(original_sample, (int, float)):
                    str_sample = str(int(original_sample))
                    if str_sample in clonefinder_data.index:
                        sample_found = True
                        sample_to_use = str_sample
                
                if sample_found and sample_to_use is not None:
                    # Get clone frequencies for this sample
                    frequencies = clonefinder_data.loc[sample_to_use].values
                    # Only consider non-zero frequencies
                    cf_nonzero_freqs = frequencies[frequencies > 0.00001]
                    
                    if len(cf_nonzero_freqs) > 0:
                        # Count non-zero clones
                        sample_metrics['CloneFinder_Num_Clones'] = len(cf_nonzero_freqs)
                        
                        # Normalize frequencies to sum to 1
                        normalized_freqs = cf_nonzero_freqs / np.sum(cf_nonzero_freqs)
                        
                        # Calculate Shannon diversity index: -sum(p * ln(p))
                        try:
                            shannon_index = -np.sum(normalized_freqs * np.log(normalized_freqs))
                            sample_metrics['CloneFinder_Shannon_Index'] = shannon_index
                        except Exception as e:
                            print(f"Error calculating CloneFinder Shannon Index: {e}")
            
            # Process LICHeE data for this sample
            if lichee_data is not None and not lichee_data.empty:
                # Try to find the sample in different formats
                sample_found = False
                sample_to_use = None
                
                # Try direct match first
                if original_sample in lichee_data.index:
                    sample_found = True
                    sample_to_use = original_sample
                # Try converting string to int
                elif isinstance(original_sample, str) and original_sample.isdigit():
                    int_sample = int(original_sample)
                    if int_sample in lichee_data.index:
                        sample_found = True
                        sample_to_use = int_sample
                # Try converting int to string
                elif isinstance(original_sample, (int, float)):
                    str_sample = str(int(original_sample))
                    if str_sample in lichee_data.index:
                        sample_found = True
                        sample_to_use = str_sample
                
                if sample_found and sample_to_use is not None:
                    # Get clone frequencies for this sample
                    frequencies = lichee_data.loc[sample_to_use].values
                    # Only consider non-zero frequencies
                    lich_nonzero_freqs = frequencies[frequencies > 0.00001]
                    
                    if len(lich_nonzero_freqs) > 0:
                        # Count non-zero clones
                        sample_metrics['LICHeE_Num_Clones'] = len(lich_nonzero_freqs)
                        
                        # Normalize frequencies to sum to 1
                        normalized_freqs = lich_nonzero_freqs / np.sum(lich_nonzero_freqs)
                        
                        # Calculate Shannon diversity index: -sum(p * ln(p))
                        try:
                            shannon_index = -np.sum(normalized_freqs * np.log(normalized_freqs))
                            sample_metrics['LICHeE_Shannon_Index'] = shannon_index
                        except Exception as e:
                            print(f"Error calculating LICHeE Shannon Index: {e}")
            
            # Add sample metrics to results dataframe
            results = pd.concat([results, pd.DataFrame([sample_metrics])], ignore_index=True)
    
    # Save results to file if requested
    saved_file_path = None
    if save_to_file and not results.empty:
        if len(patient_ids) == 1:
            # Single patient - save in patient directory
            patient_id = patient_ids[0]
            patient_dir = os.path.join(results_dir, patient_id)
            saved_file_path = os.path.join(patient_dir, f"{patient_id}_diversity_metrics.csv")
        else:
            saved_file_path = os.path.join(results_dir, "all_patients_diversity_metrics.csv")
        
        results.to_csv(saved_file_path, index=False)
        print(f"Diversity metrics saved to: {saved_file_path}")
    
    if save_to_file:
        return results, saved_file_path
    else:
        return results

In [ ]:
patient_ids = get_all_patient_ids() 

# patient_ids = ['P158']
# Calculate diversity metrics
diversity_metrics = calculate_diversity_metrics_per_sample(patient_ids)

# Display the results table
display(diversity_metrics)

In [442]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# # Load the diversity metrics data
# data = pd.read_csv('results/all_patients_diversity_metrics.csv')

# # Display basic statistics about the data
# print("Dataset summary:")
# print(f"Total samples: {len(data)}")
# print(f"Number of patients: {data['Patient_ID'].nunique()}")
# print(f"Samples per patient (min, mean, max): {data.groupby('Patient_ID').size().min()}, {data.groupby('Patient_ID').size().mean():.2f}, {data.groupby('Patient_ID').size().max()}")

# # Prepare a figure for visualizing results
# plt.figure(figsize=(15, 10))

# # 1. Distribution of number of clones detected by each tool
# plt.subplot(2, 2, 1)
# clone_data = data[['PyClone_Num_Clones', 'CloneFinder_Num_Clones', 'LICHeE_Num_Clones']].melt()
# sns.boxplot(x='variable', y='value', data=clone_data)
# plt.title('Number of Clones by Tool')
# plt.xlabel('Tool')
# plt.ylabel('Number of Clones')
# plt.xticks(rotation=45)

# # 2. Distribution of Shannon Index by each tool
# plt.subplot(2, 2, 2)
# shannon_data = data[['PyClone_Shannon_Index', 'CloneFinder_Shannon_Index', 'LICHeE_Shannon_Index']].melt()
# sns.boxplot(x='variable', y='value', data=shannon_data)
# plt.title('Shannon Index by Tool')
# plt.xlabel('Tool')
# plt.ylabel('Shannon Index')
# plt.xticks(rotation=45)

# # 3. Tool correlation
# plt.subplot(2, 2, 3)
# # Calculate correlation matrix for number of clones
# clone_corr = data[['PyClone_Num_Clones', 'CloneFinder_Num_Clones', 'LICHeE_Num_Clones']].corr()
# sns.heatmap(clone_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Correlation: Number of Clones')

# # 4. Tool correlation for Shannon Index
# plt.subplot(2, 2, 4)
# shannon_corr = data[['PyClone_Shannon_Index', 'CloneFinder_Shannon_Index', 'LICHeE_Shannon_Index']].corr()
# sns.heatmap(shannon_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Correlation: Shannon Index')

# plt.tight_layout()
# plt.savefig('diversity_overview.png')

# # Compare tools more directly with scatter plots
# plt.figure(figsize=(15, 5))

# # PyClone vs CloneFinder (Number of Clones)
# plt.subplot(1, 3, 1)
# plt.scatter(data['PyClone_Num_Clones'], data['CloneFinder_Num_Clones'], alpha=0.5)
# plt.xlabel('PyClone Num Clones')
# plt.ylabel('CloneFinder Num Clones')
# plt.title('PyClone vs CloneFinder (Num Clones)')
# plt.grid(True, alpha=0.3)

# # PyClone vs LICHeE (Number of Clones)
# plt.subplot(1, 3, 2)
# plt.scatter(data['PyClone_Num_Clones'], data['LICHeE_Num_Clones'], alpha=0.5)
# plt.xlabel('PyClone Num Clones')
# plt.ylabel('LICHeE Num Clones')
# plt.title('PyClone vs LICHeE (Num Clones)')
# plt.grid(True, alpha=0.3)

# # CloneFinder vs LICHeE (Number of Clones)
# plt.subplot(1, 3, 3)
# plt.scatter(data['CloneFinder_Num_Clones'], data['LICHeE_Num_Clones'], alpha=0.5)
# plt.xlabel('CloneFinder Num Clones')
# plt.ylabel('LICHeE Num Clones')
# plt.title('CloneFinder vs LICHeE (Num Clones)')
# plt.grid(True, alpha=0.3)

# plt.tight_layout()
# plt.savefig('tool_comparisons.png')

# # Sample-level comparisons (variability within patients)
# # Select a few patients with multiple samples for detailed analysis
# patients_with_multiple_samples = data.groupby('Patient_ID').filter(lambda x: len(x) > 2)['Patient_ID'].unique()

# if len(patients_with_multiple_samples) > 0:
#     # Select the first 3 (or fewer) patients with multiple samples
#     selected_patients = patients_with_multiple_samples[:min(3, len(patients_with_multiple_samples))]
    
#     plt.figure(figsize=(15, 5 * len(selected_patients)))
    
#     for i, patient in enumerate(selected_patients):
#         patient_data = data[data['Patient_ID'] == patient]
        
#         # Plot Number of Clones for this patient across samples
#         plt.subplot(len(selected_patients), 2, 2*i + 1)
#         patient_clone_data = patient_data[['Sample_ID', 'PyClone_Num_Clones', 'CloneFinder_Num_Clones', 'LICHeE_Num_Clones']].melt(id_vars='Sample_ID')
#         sns.barplot(x='Sample_ID', y='value', hue='variable', data=patient_clone_data)
#         plt.title(f'Patient {patient}: Number of Clones by Sample')
#         plt.xlabel('Sample')
#         plt.ylabel('Number of Clones')
#         plt.xticks(rotation=45)
#         plt.legend(title='Tool')
        
#         # Plot Shannon Index for this patient across samples
#         plt.subplot(len(selected_patients), 2, 2*i + 2)
#         patient_shannon_data = patient_data[['Sample_ID', 'PyClone_Shannon_Index', 'CloneFinder_Shannon_Index', 'LICHeE_Shannon_Index']].melt(id_vars='Sample_ID')
#         sns.barplot(x='Sample_ID', y='value', hue='variable', data=patient_shannon_data)
#         plt.title(f'Patient {patient}: Shannon Index by Sample')
#         plt.xlabel('Sample')
#         plt.ylabel('Shannon Index')
#         plt.xticks(rotation=45)
#         plt.legend(title='Tool')
    
#     plt.tight_layout()
#     plt.savefig('patient_sample_comparisons.png')

# # Identify samples with unusually high or low diversity
# # Create a function to identify outliers
# def identify_outliers(df, column, threshold=1.5):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - threshold * IQR
#     upper_bound = Q3 + threshold * IQR
    
#     outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
#     return outliers

# # Identify outliers in diversity metrics
# outliers_pyclone = identify_outliers(data, 'PyClone_Shannon_Index')
# outliers_clonefinder = identify_outliers(data, 'CloneFinder_Shannon_Index')
# outliers_lichee = identify_outliers(data, 'LICHeE_Shannon_Index')

# print("\nOutlier analysis:")
# print(f"PyClone Shannon Index outliers: {len(outliers_pyclone)}")
# print(f"CloneFinder Shannon Index outliers: {len(outliers_clonefinder)}")
# print(f"LICHeE Shannon Index outliers: {len(outliers_lichee)}")

# # Create a consensus score (average normalized diversity across tools)
# # First, create normalized versions of each metric
# for tool in ['PyClone', 'CloneFinder', 'LICHeE']:
#     for metric in ['Num_Clones', 'Shannon_Index']:
#         col = f"{tool}_{metric}"
#         # Skip columns with all NaN values
#         if not data[col].isna().all():
#             min_val = data[col].min()
#             max_val = data[col].max()
#             if max_val > min_val:  # Avoid division by zero
#                 data[f"{col}_normalized"] = (data[col] - min_val) / (max_val - min_val)

# # Calculate consensus scores (mean of normalized values)
# data['Consensus_Num_Clones'] = data[[col for col in data.columns if 'Num_Clones_normalized' in col]].mean(axis=1)
# data['Consensus_Shannon_Index'] = data[[col for col in data.columns if 'Shannon_Index_normalized' in col]].mean(axis=1)

# # Create a visualization of the consensus scores
# plt.figure(figsize=(12, 8))

# # Rank patients by average consensus diversity
# patient_avg_diversity = data.groupby('Patient_ID')['Consensus_Shannon_Index'].mean().sort_values(ascending=False)
# patient_order = patient_avg_diversity.index

# # Filter to patients with data from all three tools
# complete_data_patients = data.dropna(subset=['PyClone_Shannon_Index', 'CloneFinder_Shannon_Index', 'LICHeE_Shannon_Index'])['Patient_ID'].unique()

# if len(complete_data_patients) > 0:
#     complete_data = data[data['Patient_ID'].isin(complete_data_patients)]
    
#     # Create a heatmap of consensus diversity by patient
#     patient_sample_pivot = complete_data.pivot_table(
#         index='Patient_ID', 
#         columns='Sample_ID', 
#         values='Consensus_Shannon_Index',
#         aggfunc='first'
#     )
    
#     plt.figure(figsize=(12, max(8, len(patient_sample_pivot) * 0.4)))
#     sns.heatmap(patient_sample_pivot, cmap='viridis', annot=True, fmt='.2f')
#     plt.title('Consensus Shannon Index by Patient and Sample')
#     plt.xlabel('Sample ID')
#     plt.ylabel('Patient ID')
#     plt.tight_layout()
#     plt.savefig('consensus_diversity_heatmap.png')

# print("\nKey insights:")
# print("1. Average number of clones detected:")
# for tool in ['PyClone', 'CloneFinder', 'LICHeE']:
#     col = f"{tool}_Num_Clones"
#     print(f"   - {tool}: {data[col].mean():.2f} clones (range: {data[col].min()}-{data[col].max()})")

# print("\n2. Average Shannon diversity index:")
# for tool in ['PyClone', 'CloneFinder', 'LICHeE']:
#     col = f"{tool}_Shannon_Index"
#     print(f"   - {tool}: {data[col].mean():.2f} (range: {data[col].min():.2f}-{data[col].max():.2f})")

# # Identify patients with highest and lowest diversity
# top_diverse_patients = patient_avg_diversity.head(3)
# bottom_diverse_patients = patient_avg_diversity.tail(3)

# print("\n3. Most heterogeneous patients (highest average Shannon Index):")
# for patient, score in top_diverse_patients.items():
#     print(f"   - Patient {patient}: {score:.2f}")

# print("\n4. Most homogeneous patients (lowest average Shannon Index):")
# for patient, score in bottom_diverse_patients.items():
#     print(f"   - Patient {patient}: {score:.2f}")

# # Agreement between tools
# print("\n5. Tool agreement analysis:")
# # Calculate percentage of samples where all tools agree on high/low diversity
# # (using median as threshold)
# median_pyclone = data['PyClone_Shannon_Index'].median()
# median_clonefinder = data['CloneFinder_Shannon_Index'].median()
# median_lichee = data['LICHeE_Shannon_Index'].median()

# data['PyClone_High'] = data['PyClone_Shannon_Index'] > median_pyclone
# data['CloneFinder_High'] = data['CloneFinder_Shannon_Index'] > median_clonefinder
# data['LICHeE_High'] = data['LICHeE_Shannon_Index'] > median_lichee

# # Calculate agreement
# complete_rows = data.dropna(subset=['PyClone_High', 'CloneFinder_High', 'LICHeE_High'])
# all_agree_high = complete_rows[complete_rows['PyClone_High'] & complete_rows['CloneFinder_High'] & complete_rows['LICHeE_High']]
# all_agree_low = complete_rows[~complete_rows['PyClone_High'] & ~complete_rows['CloneFinder_High'] & ~complete_rows['LICHeE_High']]

# print(f"   - Samples where all tools agree on high diversity: {len(all_agree_high)} ({len(all_agree_high)/len(complete_rows)*100:.1f}%)")
# print(f"   - Samples where all tools agree on low diversity: {len(all_agree_low)} ({len(all_agree_low)/len(complete_rows)*100:.1f}%)")
# print(f"   - Samples with mixed results: {len(complete_rows) - len(all_agree_high) - len(all_agree_low)} ({(len(complete_rows) - len(all_agree_high) - len(all_agree_low))/len(complete_rows)*100:.1f}%)")